# Reproducibility Notebook for 'Massively multiplex single-molecule oligonucleosome footprinting' (Abdulhay, McNally et al)

(Notebook Authors: CP McNally & V Ramani; updated 12/07/20)

Included here are scripts & code-blocks necessary for regenerating the figures shown in Abdulhay, McNally et al (https://doi.org/10.7554/eLife.59404). Raw and processed data are available on GEO (Accession GSE162410), processed data and some summaries can be found on Zenodo (https://doi.org/10.5281/zenodo.3834705). The code is a mix of Python and R, and is marked accordingly.

# Figure 1: *In vitro* nucleosome array analyses 
### Supplementary Figures 2, 3

The first cell contains python code that generates a CSV containing information various measurements of the mean IPD for each molecule. The second cell contains R code that uses this CSV to generate Supplementary Figures 2 and 3

In [ ]:
# Get information from each molecule in Sequel 1/2 chromatin and controls, for paper Sup Fig 2A,B
# 4/15/2020

from __future__ import print_function, division
import sys, os
import numpy as np
import pandas as pd
import pbcore.io as pb
from pbcore.sequence import reverseComplement
from Bio import Seq, SeqIO
from tqdm import tqdm
import edlib
import warnings

def extractIPDarray(cbamFile, bamFile):
    bam = pb.IndexedBamReader(bamFile)
    cbam = pb.IndexedBamReader(cbamFile)
    
    #Find bases that are A or T
    refisa = np.array([b == 'A' for b in refseq])
    refisc = np.array([b == 'C' for b in refseq])
    refisg = np.array([b == 'G' for b in refseq])
    refist = np.array([b == 'T' for b in refseq])
    
    #find all ZMW with ccs that pass filter
    validzmw = []
    for cc in cbam:
        validzmw.append(cc.HoleNumber)

    dicdat = {'ZMW':[], 'nSubreads':[], 'nForwardSubreads':[], 'nReverseSubreads':[], 'relCCSLength':[],
              'ccsEditDistance':[], 'meanIPD':[], 'meanForwardIPD':[], 'meanReverseIPD':[], 
              'meanIPDA':[], 'meanIPDC':[], 'meanIPDG':[], 'meanIPDT':[],
              'normMeanIPD':[], 'normMeanForwardIPD':[], 'normMeanReverseIPD':[], 
              'normMeanIPDA':[], 'normMeanIPDC':[], 'normMeanIPDG':[], 'normMeanIPDT':[],
              'widmeanIPD':[], 'widmeanForwardIPD':[], 'widmeanReverseIPD':[], 
              'widmeanIPDA':[], 'widmeanIPDC':[], 'widmeanIPDG':[], 'widmeanIPDT':[],
              'widnormMeanIPD':[], 'widnormMeanForwardIPD':[], 'widnormMeanReverseIPD':[], 
              'widnormMeanIPDA':[], 'widnormMeanIPDC':[], 'widnormMeanIPDG':[], 'widnormMeanIPDT':[],
              'outmeanIPD':[], 'outmeanForwardIPD':[], 'outmeanReverseIPD':[], 
              'outmeanIPDA':[], 'outmeanIPDC':[], 'outmeanIPDG':[], 'outmeanIPDT':[],
              'outnormMeanIPD':[], 'outnormMeanForwardIPD':[], 'outnormMeanReverseIPD':[], 
              'outnormMeanIPDA':[], 'outnormMeanIPDC':[], 'outnormMeanIPDG':[], 'outnormMeanIPDT':[]}
        
    for zind, zmw in enumerate(tqdm(validzmw, position=0, desc='Getting IPDs')):
        cc = cbam.readsByHoleNumber(zmw)
        if not len(cc) == 1:
            print('cc len: %d' % (len(cc)))
        cc = cc[0]
        forwardCCS = cc.read(aligned=False)
        reverseCCS = reverseComplement(forwardCCS)
        faln = edlib.align(forwardCCS, refseq, mode='NW', task='path')
        raln = edlib.align(reverseCCS, refseq, mode='NW', task='path')
        
        subrs = bam.readsByHoleNumber(zmw)
        if len(subrs) == 0:
            continue
        allipds = np.empty((len(subrs), len(refseq)))
        allipds.fill(np.nan)
        subrOrient = np.empty(len(subrs))

        for index, sr in enumerate(subrs):
            subrOrient[index] = sr.isForwardStrand
            cigarv = sr.unrolledCigar(orientation="genomic")
            #using subread base calls that are matches or mismatches
            #may want to consider ignoring mismatches in the future, especially for non-synethetic source DNA
            #insertions = 1, gaps = 2, leaving both out of downstream analysis
            usebases =  cigarv == 7 #np.logical_or(cigarv == 7, cigarv == 8)
            ipds = sr.baseFeature('Ipd',aligned=True, orientation="genomic")
            pws = sr.baseFeature('PulseWidth',aligned=True, orientation="genomic")
            refpos = sr.referencePositions(aligned=True, orientation="genomic")
            #use the reference positions to index the per base IPD values into their position in the reference sequence
            allipds[index, refpos[usebases]] = ipds[usebases]
        normallipds = allipds / np.mean(np.percentile(allipds[~np.isnan(allipds)],usepercentiles))
        
        # I expect to see RuntimeWarnings in this block
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            formean = np.nanmean(allipds[subrOrient == True,], axis=0)
            revmean = np.nanmean(allipds[subrOrient == False,], axis=0)
            normformean = np.nanmean(normallipds[subrOrient == True,], axis=0)
            normrevmean = np.nanmean(normallipds[subrOrient == False,], axis=0)
        
        
        dicdat['ZMW'].append(zmw)
        dicdat['nSubreads'].append(len(subrs))
        dicdat['nForwardSubreads'].append(np.sum(subrOrient == True))
        dicdat['nReverseSubreads'].append(np.sum(subrOrient == False))
        dicdat['relCCSLength'].append(cc.readLength - len(refseq))
        dicdat['ccsEditDistance'].append(min(faln['editDistance'], raln['editDistance']))
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            dicdat['meanIPD'].append(np.nanmean(np.concatenate([formean, revmean])))
            dicdat['meanForwardIPD'].append(np.nanmean(formean))
            dicdat['meanReverseIPD'].append(np.nanmean(revmean))
            dicdat['meanIPDA'].append(np.nanmean(np.concatenate([formean[refisa], revmean[refist]])))
            dicdat['meanIPDC'].append(np.nanmean(np.concatenate([formean[refisc], revmean[refisg]])))
            dicdat['meanIPDG'].append(np.nanmean(np.concatenate([formean[refisg], revmean[refisc]])))
            dicdat['meanIPDT'].append(np.nanmean(np.concatenate([formean[refist], revmean[refisa]])))
            dicdat['normMeanIPD'].append(np.nanmean(np.concatenate([normformean, normrevmean])))
            dicdat['normMeanForwardIPD'].append(np.nanmean(normformean))
            dicdat['normMeanReverseIPD'].append(np.nanmean(normrevmean))
            dicdat['normMeanIPDA'].append(np.nanmean(np.concatenate([normformean[refisa], normrevmean[refist]])))
            dicdat['normMeanIPDC'].append(np.nanmean(np.concatenate([normformean[refisc], normrevmean[refisg]])))
            dicdat['normMeanIPDG'].append(np.nanmean(np.concatenate([normformean[refisg], normrevmean[refisc]])))
            dicdat['normMeanIPDT'].append(np.nanmean(np.concatenate([normformean[refist], normrevmean[refisa]])))
            # now within widom
            widfmean = formean[chrUseb == 1]
            widrmean = revmean[chrUseb == 1]
            normwidfmean = normformean[chrUseb == 1]
            normwidrmean = normrevmean[chrUseb == 1]
            widisa = refisa[chrUseb == 1]
            widisc = refisc[chrUseb == 1]
            widisg = refisg[chrUseb == 1]
            widist = refist[chrUseb == 1]
            dicdat['widmeanIPD'].append(np.nanmean(np.concatenate([widfmean, widrmean])))
            dicdat['widmeanForwardIPD'].append(np.nanmean(widfmean))
            dicdat['widmeanReverseIPD'].append(np.nanmean(widrmean))
            dicdat['widmeanIPDA'].append(np.nanmean(np.concatenate([widfmean[widisa], widrmean[widist]])))
            dicdat['widmeanIPDC'].append(np.nanmean(np.concatenate([widfmean[widisc], widrmean[widisg]])))
            dicdat['widmeanIPDG'].append(np.nanmean(np.concatenate([widfmean[widisg], widrmean[widisc]])))
            dicdat['widmeanIPDT'].append(np.nanmean(np.concatenate([widfmean[widist], widrmean[widisa]])))
            dicdat['widnormMeanIPD'].append(np.nanmean(np.concatenate([normwidfmean, normwidrmean])))
            dicdat['widnormMeanForwardIPD'].append(np.nanmean(normwidfmean))
            dicdat['widnormMeanReverseIPD'].append(np.nanmean(normwidrmean))
            dicdat['widnormMeanIPDA'].append(np.nanmean(np.concatenate([normwidfmean[widisa], normwidrmean[widist]])))
            dicdat['widnormMeanIPDC'].append(np.nanmean(np.concatenate([normwidfmean[widisc], normwidrmean[widisg]])))
            dicdat['widnormMeanIPDG'].append(np.nanmean(np.concatenate([normwidfmean[widisg], normwidrmean[widisc]])))
            dicdat['widnormMeanIPDT'].append(np.nanmean(np.concatenate([normwidfmean[widist], normwidrmean[widisa]])))
            # now outside widom
            outfmean = formean[chrUseb == 2]
            outrmean = revmean[chrUseb == 2]
            normoutfmean = normformean[chrUseb == 2]
            normoutrmean = normrevmean[chrUseb == 2]
            outisa = refisa[chrUseb == 2]
            outisc = refisc[chrUseb == 2]
            outisg = refisg[chrUseb == 2]
            outist = refist[chrUseb == 2]
            dicdat['outmeanIPD'].append(np.nanmean(np.concatenate([outfmean, outrmean])))
            dicdat['outmeanForwardIPD'].append(np.nanmean(outfmean))
            dicdat['outmeanReverseIPD'].append(np.nanmean(outrmean))
            dicdat['outmeanIPDA'].append(np.nanmean(np.concatenate([outfmean[outisa], outrmean[outist]])))
            dicdat['outmeanIPDC'].append(np.nanmean(np.concatenate([outfmean[outisc], outrmean[outisg]])))
            dicdat['outmeanIPDG'].append(np.nanmean(np.concatenate([outfmean[outisg], outrmean[outisc]])))
            dicdat['outmeanIPDT'].append(np.nanmean(np.concatenate([outfmean[outist], outrmean[outisa]])))
            dicdat['outnormMeanIPD'].append(np.nanmean(np.concatenate([normoutfmean, normoutrmean])))
            dicdat['outnormMeanForwardIPD'].append(np.nanmean(normoutfmean))
            dicdat['outnormMeanReverseIPD'].append(np.nanmean(normoutrmean))
            dicdat['outnormMeanIPDA'].append(np.nanmean(np.concatenate([normoutfmean[outisa], normoutrmean[outist]])))
            dicdat['outnormMeanIPDC'].append(np.nanmean(np.concatenate([normoutfmean[outisc], normoutrmean[outisg]])))
            dicdat['outnormMeanIPDG'].append(np.nanmean(np.concatenate([normoutfmean[outisg], normoutrmean[outisc]])))
            dicdat['outnormMeanIPDT'].append(np.nanmean(np.concatenate([normoutfmean[outist], normoutrmean[outisa]])))
              
    sampdf = pd.DataFrame(dicdat)
    return(sampdf)

os.chdir('~/pbanalysis')
sampleRef = pd.read_csv('/avicenna/vramani/analyses/pacbio/pbrun345_SampleReference.csv')
usepercentiles = range(10,41)
for ir, record in enumerate(SeqIO.parse(sampleRef.reference[0], 'fasta')):
        if ir > 0:
            raise InputError('Reference fasta has multiple entries')
        refseq = record.seq

# bases containing repeats of the widom 601 sequence are labeled with 1
chrUseb = np.empty(len(refseq))
chrUseb[0:1735] = 2
chrUseb[23:170] = 1
chrUseb[216-363] = 1
chrUseb[409:556] = 1
chrUseb[602:749] = 1
chrUseb[795:942] = 1
chrUseb[983:1130] = 1
chrUseb[1180:1326] = 1
chrUseb[1372:1519] = 1
chrUseb[1565:1712] = 1



comparesamps = [0,1,2,3, 29, 30]
unifiedSampleName = {0:'Chromatin', 1:'Naked Methylated', 2:'Naked Negative',
                     3:'Chromatin', 30:'Naked Methylated', 29:'Naked Negative'}
sampDFs = []
for samp in comparesamps:
    sdf = extractIPDarray(sampleRef.ccsFile[samp], sampleRef.alignedSubreadsFile[samp])
    sdf['sample'] = unifiedSampleName[samp]
    sdf['cell'] = sampleRef.sampleName[samp]
    if samp <= 2:
        sdf['machine'] = 1
    else:
        sdf['machine'] = 2
    sampDFs.append(sdf)

totdf = pd.concat(sampDFs)
#totdf.to_pickle('processed/forFigures/meanIPDinfoChrControls.pickle', protocol=-1)
totdf.to_csv('processed/forFigures/meanIPDinfoChrControls.csv', index=False, float_format='%.4f')

In [ ]:
library(ggplot2)
library(tidyr)
setwd('~/pbanalysis')
ipddf = read.csv("processed/forFigures/meanIPDinfoChrControls.csv")
ipddf$sample = factor(ipddf$sample, levels=c('Naked Negative', 'Naked Methylated', 'Chromatin')) #changing order


filtercriteria = abs(ipddf$relCCSLength) <= 2 & ipddf$nForwardSubreads >= 3 & ipddf$nReverseSubreads >= 3

sprintf('Before filtering, %d Sequel molecules, %d Sequel II molecules, %d total', sum(ipddf$machine == 1),
        sum(ipddf$machine == 2), dim(ipddf)[1])
sprintf('After filtering, %d Sequel molecules, %d Sequel II molecules, %d total', sum(filtercriteria & ipddf$machine == 1),
        sum(filtercriteria & ipddf$machine == 2), sum(filtercriteria))

# Supplementary Figure 2
ipddfsub = ipddf[filtercriteria, c("meanIPD", "normMeanIPD", "machine", "sample")]
# rescale mean normalized IPD to put it back on the same scale as the mean raw IPD
ipddfsub$normMeanIPD = ipddfsub$normMeanIPD * (mean(ipddfsub$meanIPD) / mean(ipddfsub$normMeanIPD))
normnot_long = pivot_longer(ipddfsub, c("meanIPD", "normMeanIPD"), names_to="normed", values_to="meanIPD")

facetlab_names = c('1'='Sequel', '2'='Sequel II', 'meanIPD'='Raw IPD', 'normMeanIPD'='Quantile Normalized')
ggplot(data = normnot_long, aes(meanIPD, colour=sample, fill=sample)) +
  geom_density(alpha=0.4) +
  facet_grid(cols=vars(machine), rows=vars(normed), labeller=as_labeller(facetlab_names), scales='free_y') +
  scale_fill_brewer(palette='Dark2', name='Sample', labels=c('Naked DNA', 'Naked DNA + EcoGII', 'Chromatin + EcoGII')) +
  scale_colour_brewer(palette='Dark2', name='Sample', labels=c('Naked DNA', 'Naked DNA + EcoGII', 'Chromatin + EcoGII')) +
  scale_x_continuous(limits=c(0,150)) +
  labs(x='Mean IPD', y='Density') +
  theme_bw()
ggsave("Figures/nearFinal/moleculeMeanIPDdist.png", width=8, height=6, dpi=300)
ggsave("Figures/nearFinal/moleculeMeanIPDdist.pdf", width=8, height=6)


# Supplementary Figure 3
ipddfsub = ipddf[filtercriteria, c("widnormMeanIPDA","widnormMeanIPDC","widnormMeanIPDG","widnormMeanIPDT",
                                               "outnormMeanIPDA","outnormMeanIPDC","outnormMeanIPDG","outnormMeanIPDT",
                                               "machine", "sample")]
ipddflong = pivot_longer(ipddfsub, c("widnormMeanIPDA","widnormMeanIPDC","widnormMeanIPDG","widnormMeanIPDT",
                                     "outnormMeanIPDA","outnormMeanIPDC","outnormMeanIPDG","outnormMeanIPDT"),
                         names_to=c("widom","base"), names_pattern="([a-z]{3})normMeanIPD([A-Z])", values_to="meanIPD")
ipddflong$base = factor(ipddflong$base, levels=c('T','G','C','A'))
facetlab_names = c('out'='Linker', 'wid'='Widom 601 Sequence', 'Naked Negative'='Naked DNA',
                   'Naked Methylated'='Naked DNA + EcoGII', 'Chromatin'='Chromatin + EcoGII')
ggplot(data = ipddflong[ipddflong$machine == 1,], aes(x=base, y=meanIPD)) +
  geom_violin(fill='rosybrown') +
  facet_grid(cols=vars(sample), rows=vars(widom), labeller=as_labeller(facetlab_names)) +
  scale_x_discrete(labels=c('A','C','G','T')) +
  labs(x="Template Base", y='Mean IPD', title='Sequel') +
  theme_bw()
ggsave("Figures/nearFinal/moleculeMeanIPDbyBase_Sequel.png", width=10, height=6, dpi=300)
ggsave("Figures/nearFinal/moleculeMeanIPDbyBase_Sequel.pdf", width=10, height=6)

ggplot(data = ipddflong[ipddflong$machine == 2,], aes(x=base, y=meanIPD)) +
  geom_violin(fill='rosybrown') +
  facet_grid(cols=vars(sample), rows=vars(widom), labeller=as_labeller(facetlab_names)) +
  scale_x_discrete(labels=c('A','C','G','T')) +
  labs(x="Template Base", y='Mean IPD', title='Sequel II') +
  theme_bw()
ggsave("Figures/nearFinal/moleculeMeanIPDbyBase_SequelII.png", width=10, height=6, dpi=300)
ggsave("Figures/nearFinal/moleculeMeanIPDbyBase_SequelII.pdf", width=10, height=6)


### Figure 1D, Supplementary Figure 5

The below code produces heatmaps of the methylation posterior probability across individual molecules. It takes as input the numpy files produced as outputs from extractIPD.py

In [ ]:
library(ggplot2)
library(tidyr)
library(RcppCNPy)
library(scales)
library(zoo)

nanmean <- function(x) {
  nonnan = x[is.finite(x)]
  if (length(nonnan) < 1) {
    return(NA)
  } else {
    return( mean(nonnan))
  }
}

plotNmolecules = 500
smoothbases = 5

basep = '~/pbanalysis/processed/binarized'
figurefolder = '~/pbanalysis/Figures/nearFinal/'
samples = c('pbrun4_gold_nuc47_chromatin', 'pbrun5_cell2_47bp_DNA_minusM_rep1', 'pbrun5_cell2_47bp_DNA_plusM_rep1')

for (samp in samples) {
  bipds = npyLoad(file.path(basep, paste(samp, '_bingmm.npy', sep='')))
  
  rolltipd = bipds[1:plotNmolecules,]
  for (i in 1:500) {
    rolltipd[i,] = rollapply(bipds[i,], smoothbases, nanmean, align='left', partial=TRUE)
  }
  
  rzdf = data.frame(rolltipd[1:plotNmolecules,])
  rzdf$molecule = 1:plotNmolecules
  rzdflong = pivot_longer(rzdf, X1:X2268, names_prefix='X', names_to='base', values_to='ipd')
  rzdflong$base = as.integer(rzdflong$base)
  ggplot(rzdflong[is.finite(rzdflong$ipd) & rzdflong$molecule <= plotNmolecules,], aes(x=base, y=molecule, fill=ipd)) +
    geom_raster() +
    scale_fill_distiller(palette='BuPu', direction=1, limits=c(0,1), oob=squish, name='Posterior probability methylated\n(5 bp smoothed)') +
    scale_x_continuous(expand=c(0,0)) +
    scale_y_continuous(expand=c(0,0)) +
    labs(x="Position along molecule", y='Molecules') +
    theme_bw()
  ggsave(file.path(figurefolder, paste('heatmapRollingpp5c_', samp, '.png',sep='')), width=6, heigh=5, dpi=300)
  ggsave(file.path(figurefolder, paste('heatmapRollingpp5c_', samp, '.pdf',sep='')), width=6, heigh=5)
}

### Figure 1B, 1C

The below code uses files output by callNucPeaks.py and calculates deviations of these nucleosome calls from the expected center of the widom 601 sequence, as well as distances between adjacent nucleosome calls.

In [ ]:
library(feather)
library(ggplot2)

setwd('~/pbanalysis')
bpc = read_feather('processed/peaks/binarypeaks/pbrun4_gold_nuc47_chromatin_peaks.feather')

widomcenters = c(23, 216, 409, 602, 795, 983, 1179, 1372, 1565) + 73
distToWidC = vector(length=dim(bpc)[1])
for (i in 1:dim(bpc)[1]) {
  dtow = bpc$pos[i] - widomcenters
  distToWidC[i] = dtow[which.min(abs(dtow))]
}
bpc$distToW = distToWidC


hist(bpc$distToW[bpc$distToW <= 100])
ggplot(bpc[bpc$distToW <= 100,], aes(distToW)) +
  geom_density(fill='black') +
  labs(x='Distance to expected dyad', y='Density') +
  theme_bw()
ggsave('Figures/nearFinal/dToExpDyad.png', width=4, height=3,units='in')
ggsave('Figures/nearFinal/dToExpDyad.pdf', width=4, height=3,units='in')
sprintf('Mean distance to expected dyad: %f', mean(bpc$distToW[bpc$distToW <= 100]))
sprintf('Standard deviation of predicted dyad position: %f', sd(bpc$distToW[bpc$distToW <= 100]))
sprintf('Mean absolute distance to expected dyad: %f', mean(abs(bpc$distToW[bpc$distToW <= 100])))

bpc$dToNext = NA
for (i in 1:(dim(bpc)[1]-1)) {
  if (bpc$zmw[i+1] == bpc$zmw[i]) {
    bpc$dToNext[i] = bpc$pos[i+1] - bpc$pos[i]
  }
}
abpc = bpc[bpc$pos <= (widomcenters[8] + (widomcenters[9] - widomcenters[8]) / 2),]
abpc = abpc[is.finite(abpc$dToNext),]

zmw = unique(abpc$zmw)
molmeandtn = vector(length=length(zmw))
for (i in 1:length(zmw)) {
  molmeandtn[i] = mean(abpc$dToNext[abpc$zmw == zmw[i]])
}

molmeand = data.frame(molmeandtn)
molmeand$type = 'molecule'
molmeand$nrl = molmeand$molmeandtn
subd = abpc$dToNext
subd = data.frame(subd)
subd$nrl = subd$subd
subd$type = 'individual'
combd = rbind(subset(subd, select=c(nrl,type)), subset(molmeand, select=c(nrl,type)))
type_names = c('individual'='Single gap', 'molecule'='Molecule mean')
ggplot(combd, aes(nrl, fill=type, color=type)) +
  geom_vline(xintercept=193) +
  geom_density(alpha=0.5) +
  scale_x_continuous(limits=c(147,275)) +
  scale_color_brewer(palette = "Set1", name='',labels = c('Single distances', 'Molecule means')) +
  scale_fill_brewer(palette = "Set1", name='',labels = c('Single distances', 'Molecule means')) +
  labs(y='Density', x='Distance between nucleosome calls') +
  theme_bw()
ggsave('Figures/nearFinal/dBetweenNucCalls.png', width=5, height=3,units='in')
ggsave('Figures/nearFinal/dBetweenNucCalls.pdf', width=5, height=3,units='in')

### Supplementary Figure 5

This code plots the distribution of nucleosome calls across the molecule as well as the average posterior probabilities of methylation.

Input: Binarized base calls output by extractIPD.py, and predicted nucleosome centers from callNucPeaks.py

In [ ]:
# Make comparison of remodeled arrays - Now with Binarized data
# Colin McNally 2020/03/02

library(RcppCNPy)
library(ggplot2)
library(RColorBrewer)
library(feather)

setwd('~/pbanalysis')

usesamples = list(c('pbrun4_gold', 'nuc47_chromatin'))
use_names = c('nuc47_chromatin' = 'Chromatin')

add_widomRect <- function(xstart){
    annotate("rect", ymin = -Inf, ymax = Inf, xmin=xstart, xmax=xstart+146, alpha = 0.7, fill=bcols[3] )
}

thesesamples = usesamples
thesenames = use_names

dfipd = data.frame()
for (samp in thesesamples) {
    allTipds = npyLoad(file.path('processed','binarized', paste(samp[1], '_', samp[2], '_bingmm.npy', sep='')))
    ipdmeans = colMeans(allTipds, na.rm=T)
    ipdsds = apply(allTipds, 2, sd, na.rm=T)
    xind = which(is.finite(ipdmeans))
    df = data.frame("pos" = xind, "means" = ipdmeans[xind], "sds" = ipdsds[xind])
    df$sample = samp[2]
    dfipd = rbind(dfipd, df)
}

dfpeaks = data.frame()
for (samp in thesesamples) {
    samppeaks = read_feather(file.path('processed','peaks','binarypeaks', paste(samp[1], '_', samp[2], '_peaks.feather', sep='')))
    samppeaks$sample = samp[2]
    dfpeaks = rbind(dfpeaks, samppeaks)
}

widomstart = c(23, 216, 409, 602, 795, 983, 1179, 1372, 1565)
bcols = brewer.pal(7, 'PuOr')

ggplot(data = dfipd, aes(x=pos)) + 
    scale_y_continuous("Called dyad density", sec.axis = sec_axis(~ ( . - .15) / .65, name = "Mean methylation call")) +
    add_widomRect(widomstart) +
    geom_histogram(data=dfpeaks, aes(y=..count../max(dfpeaks$zmw)), binwidth=5, fill=bcols[1]) +
    geom_point(y=.15 + dfipd$means * .65, size=1, shape=19, color=bcols[7], alpha=1, stroke=0) +
    xlab('Position along molecule') + 
    theme_bw()

fname = 'BinarizedPeakCallWholeMolecule'
ggsave(paste("Figures/nearFinal/", fname,'.pdf', sep=''), width=9.5,height=3,units="in")
ggsave(paste("Figures/nearFinal/", fname,'.png', sep=''), width=9.5,height=3,dpi=300,units="in")

### Supplementary Figure 4

This figure shows the bias in methylation predictions at different sequence contexts

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib as mpl
from Bio import Seq, SeqIO, SeqRecord
import os

from tqdm.notebook import tqdm

os.chdir('/avicenna/cmcnally/pbanalysis')

sampleRef = pd.read_csv('/avicenna/vramani/analyses/pacbio/pbrun3-8c2_SampleReference.csv')
for record in SeqIO.parse(sampleRef['reference'][0], 'fasta'):
    refseq = record.seq
    
# set Arial as the default sans-serif font
mpl.rcParams['font.sans-serif'] = "Arial"
# Then, "ALWAYS use sans-serif fonts"
mpl.rcParams['font.family'] = "sans-serif"

usesamples = [29, 30]
tipds = {}
bipds = {}
lipds = {}
for isamp, samp in enumerate(usesamples):
    tipds[samp] = np.load(os.path.join('processed','onlyT',
                                 sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_onlyT.npy'))
    bipds[samp] = np.load(os.path.join('processed','binarized',
                             sampleRef['cell'][samp] + '_' + sampleRef['sampleName'][samp] + '_bingmm.npy'))
    lipds[samp] = np.log10(tipds[samp])
bisa = np.nonzero(np.sum(~np.isnan(tipds[30]), 0) > 0)[0]

# final version using -2:+7

kmerlen = 10
basecomplement = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}
basesame = {'A':'A', 'C':'C', 'G':'G', 'T':'T'}
seqMatrix = np.full((len(bisa), kmerlen), '-', dtype=np.dtype('U1'))

for ib, base in enumerate(bisa):
    for offset in np.arange(-2,8):
        if refseq[base] == 'A':
            pos = base + offset
            usedic = basesame
        if refseq[base] == 'T':
            pos = base - offset
            usedic = basecomplement
        if pos >= 0 and pos < len(refseq):
            seqMatrix[ib, offset + 2] = usedic[refseq[pos][0]]

posGrouped = []
for ib, base in enumerate(bisa):
    if np.sum(seqMatrix[ib,:] == '-') == 0:
        if len(posGrouped) == 0:
            posGrouped.append([ib])
        else:
            foundExistingGroup = False
            for grp in range(len(posGrouped)):
                if np.sum(seqMatrix[ib,:] == seqMatrix[posGrouped[grp][0],:]) == seqMatrix.shape[1]:
                    posGrouped[grp].append(ib)
                    foundExistingGroup = True
                    break
            if not foundExistingGroup:
                posGrouped.append([ib])
                
vseqdict = {'A':3, 'C':2, 'G':1, 'T':0}
vseqMat = np.full((len(posGrouped), kmerlen), '0', np.int8)
for grp in range(len(posGrouped)):
    for x in range(seqMatrix.shape[1]):
        vseqMat[grp,x] = vseqdict[seqMatrix[posGrouped[grp][0],x]]

cmaplist = [(251/255, 65/255, 53/255, 1),
            (18/255, 18/255, 18/255, 1),
            (65/255, 105/255, 225/255, 1),
            (50/255, 205/255, 51/255, 1)]
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, 4)

lbins = np.linspace(0, 2.2, 101)
bhist = {s:np.full((len(posGrouped), 100), 0.0) for s in usesamples}
for samp in usesamples:
    for ig, group in enumerate(posGrouped):
        h, _ = np.histogram(np.concatenate([lipds[samp][:,bisa[ib]] for ib in group]), lbins, density=True)
        bhist[samp][ig,:] = h

combhist = np.full((len(posGrouped), 100), 0.0)
for ig, group in enumerate(posGrouped):
        h, _ = np.histogram(np.concatenate([lipds[samp][:,bisa[ib]] for ib in group for samp in [29,30]]), lbins, density=True)
        combhist[ig,:] = h

fracmeth = np.full((len(posGrouped)), np.nan)
for ig, group in enumerate(posGrouped):
    thisbasebin = np.concatenate([bipds[samp][:,bisa[ib]] for ib in group for samp in [29,30]])
    fracmeth[ig] = np.sum(thisbasebin > 0.5) / np.sum(~np.isnan(thisbasebin))
sortby = np.argsort(fracmeth)

fig, ax = plt.subplots(1,4, figsize=(9,8), gridspec_kw={'width_ratios':[1,1,1.25,1.2]})
ax[0].imshow(bhist[29][sortby,:], aspect='auto', extent=[0, 2, len(posGrouped), 0], vmin=0, vmax=4)

im1 = ax[1].imshow(combhist[sortby,:], aspect='auto', extent=[0, 2, len(posGrouped), 0], vmin=0, vmax=4)
#cb1 = fig.colorbar(im1, ax=ax[1], aspect=10, label='Density')
im2 = ax[2].imshow(bhist[30][sortby,:], aspect='auto', extent=[0, 2, len(posGrouped), 0], vmin=0, vmax=4)
cb2 = fig.colorbar(im2, ax=ax[2], aspect=10, label='Density')
cb2.set_ticks([0,1,2,3,4])

im3 = ax[3].imshow(vseqMat[sortby,:], aspect='auto', cmap=cmap, interpolation='nearest', vmin=-0.5, vmax=3.5)
cb = fig.colorbar(im3, ax=ax[3], aspect=10)
cb.set_ticks([0,1,2,3])
cb.set_ticklabels(['T','G','C','A'])
ax[0].set_title('Negative Control')
ax[1].set_title('Combined')
ax[2].set_title('Positive Control')
ax[3].set_title('Sequence Context')
ax[0].set_ylabel('%d-mers' % (kmerlen))
for i in range(3):
    ax[i].set_xticks([0,1,2])
    ax[i].set_xticklabels([1, 10, 100])
    ax[i].set_xlabel('Normalized IPD')
ax[3].set_xticks([0,2,5,9])
ax[3].set_xticklabels([-2, 0,3,7])
#plt.subplots_adjust(wspace=0.3)
plt.tight_layout(0.4)

fig.savefig('Figures/Revision/widom_%dmerDistributions_byCombBinF.png' % (kmerlen), dpi=300)
fig.savefig('Figures/Revision/widom_%dmerDistributions_byCombBinF.pdf' % (kmerlen))

#plt.close()

# Figures 2 - 5: *In vivo* data analyses

There are a few hard-coded parameters here that we mention in the Methods section but would also like to call out here.


1. We smooth the posterior probabilities a few different ways during the paper to account for regions with low local A/T content and generally denoise the single-molecule signal. For in vitro analyses, we smooth the calculated posterior probabilities using a 5 bp rolling mean. For all in vivo analyses in the paper that involve calculation of single-molecule autocorrelograms, averaging over multiple templates, and visualizing individual molecules, we smooth posteriors with a 33 bp rolling mean. 


2. For all autocorrelation calculations we ignore regions where compared lengths would be unequal; this has the effect of rendering the returned autocorrelogram exactly 0.5 * the input length.

## These are helpful functions + modules that are used in the below cells.

In [ ]:
import scanpy
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy
from pylab import *
from scipy import signal
import pickle
import seaborn as sns

def ccf(x, y):
    result = np.correlate(y - np.mean(y), x - np.mean(x), 'same') / (np.std(y) * np.std(x) * len(y))
    length = (len(result) - 1) // 2
    return result[length:]

def xcor(a,b, maxlen = 1000, lengths=False):
    foo = []
    if isinstance(lengths, np.ndarray):
        for i in range(len(a)):
            res = ccf(a[i][:lengths[i]],b[i][:lengths[i]]) #take the nonNan portion of the read
            if len(res) < maxlen:
                filler = np.empty(maxlen - len(res))
                filler[:] = np.nan
                res = np.append(res, filler)
                foo.append(res)
            else:
                res = ccf(a[i],b[i])
                foo.append(res)                
    else:
        for i in range(len(a)):
            res = ccf(a[i],b[i])
            foo.append(res)
    a = None
    b = None
    return foo

def eat_pickle(pick, zmwinfo, clean_cutoff=1.02):
    '''Given a cutoff to exclude unmethylated templates, convert pickle-format ZMW info into a zmw_dict
    and return valid ZMWs + the whole dict.'''
    valid_zmws = {}
    with open(pick, 'rb') as fout:
        tipds = pickle.load(fout, encoding="latin1")
    with open(zmwinfo, 'rb') as fout:
        zmws = pickle.load(fout, encoding="latin1")
    subset = zmws[zmws['basemeanT'] / zmws['basemeanA'] >= clean_cutoff]
    for i in subset['zmw']:
        valid_zmws[i] = True
    return (tipds, valid_zmws)

def return_ipd_ratios(zmwinfo):
    with open(zmwinfo, 'rb') as fout:
        zmws = pickle.load(fout, encoding='latin1')
    ratios = zmws['basemeanT'] / zmws['basemeanA']
    return ratios

def return_probs(zmwinfo):
    prob_tot = []
    with open(zmwinfo, 'rb') as fout:
        tipds = pickle.load(fout, encoding="latin1")
    for hole in tipds:
        tot = len(tipds[hole][~np.isnan(tipds[hole])])
        frac_meth = len(tipds[hole][(tipds[hole] >= 0.25) & (~np.isnan(tipds[hole]))]) / tot
        prob_tot.append(frac_meth)
    return prob_tot

def eat_pickle_binary(pick, samp_label):
    with open(pick, 'rb') as fout:
        tipds = pickle.load(fout, encoding="latin1")    
    return (tipds)

def eat_pickle_b2m(pick, samp_label,length, min_len = 500, samp= 0):
    with open(pick, 'rb') as fout:
        tipds = pickle.load(fout, encoding="latin1")    
    nmol = 0
    lengths = []
    reads_f = []
    labels = []
    holes = []
#     reads_r = []
    mean_probs = []
    index = 0
    for hole in tipds:
        read = tipds[int(hole)]
#         read_r = read[::-1]
        if len(read) < min_len: continue
        labels.append(samp_label)
        holes.append("%s_%s" % (samp_label, hole))
        if len(tipds[int(hole)]) >= length:
            reads_f.append(read[:length])
#             reads_r.append(read_r[:length])
            lengths.append(len(tipds[int(hole)]))
#            holes.append(rep_label)
        elif len(tipds[int(hole)]) < length:
            filler = np.empty(length - len(tipds[int(hole)]))
            filler[:] = np.nan
            lengths.append(len(read))
            read = np.append(read, filler)
            reads_f.append(read)
        nmol += 1
        index += 1
        if samp != 0:
            if nmol == samp: break
    new_mat = np.vstack(reads_f)
#     new_mat_r = np.vstack(reads_r)
    return (new_mat, lengths, labels, holes)
    
def distill_tipds_bed(tipds, valid_zmws, sitelist, length, label, subsample=0,min_len=500):
    '''Used to extract relevant mapped ZMWs given a list of ZMW ids. Also excludes bad ZMWs given the valid
    ZMW list. This is for fragment ends (doesn't search for a flat feaure of interest).'''
    sites = open(sitelist,'r')
    hole_nos = {}
    invalid = 0
    tot = 0
    for i in sites:
        split = i.split()
        tot += 1
        if int(split[0]) not in valid_zmws: 
            invalid +=1
            continue
        hole_nos[split[0]] = [split[1], int(split[3]) - int(split[2]), split[4]]
    sites.close()
    new_mat = np.zeros(length)
    lengths = []
    labs = []
    reads_f = []
    reads_r = []
    n_mol = 0
    holes = []
    for hole in hole_nos:
        if subsample != 0:
            if n_mol >= subsample: break
        read = tipds[int(hole)]
        read_r = read[::-1]
        dist = hole_nos[hole]
        if len(tipds[int(hole)]) < min_len: continue
        if len(tipds[int(hole)]) >= length:
            reads_f.append(read[:length])
            reads_r.append(read_r[:length])
            lengths.append(len(tipds[int(hole)]))
            labs.append(dist[-1])
#            holes.append(rep_label)
            n_mol +=1
        elif len(tipds[int(hole)]) < length:
            filler = np.empty(length - len(tipds[int(hole)]))
            filler[:] = np.nan
            lengths.append(len(read))
            read = np.append(read, filler)
            read_r = np.append(read_r, filler)
            reads_f.append(read)
            reads_r.append(read_r)
            labs.append(dist[-1])
#            holes.append(rep_label)
            n_mol +=1
    new_mat = np.vstack(reads_f)
    new_mat_r = np.vstack(reads_r)
    dinuc_mat = np.vstack(dinucs)
    return (new_mat, new_mat_r, lengths, labs,holes,invalid,tot)

def distill_tipds_flat_centered(tipds, sitelist, length, label,subsample=0):
    sites = open(sitelist,'r')
    hole_nos = {}
    for i in sites:
        split = i.split()
        #check to make sure the feature is in the middle of the read 
        #and enough on both sites
        if int(split[2]) <= int(split[4]) <= int(split[3]):
            index1 = int(split[4]) - int(split[2])
            index2 = int(split[3]) - int(split[4])
            strand = split[-1]
            r_strand = split[-2]
            if index1 > (length / 2) and index2 > (length / 2):
                hole_nos[split[0]] = (index1, strand, r_strand)
    sites.close()
    lengths = []
    labs = []
    reads = []
    mno = 0
    for hole in hole_nos:
        read = tipds[int(hole)]
        index,strand,r_strand = hole_nos[hole]
        if r_strand == '-':
            read = read[::-1]
        if strand == "+":
            extract = tipds[int(hole)][int(index - (length / 2)): int(index + (length / 2))]
            if len(extract) != length: continue
            reads.append(extract)
            labs.append(label)
            lengths.append(len(tipds[int(hole)]))
        else:
            extract = tipds[int(hole)][::-1][int(index - (length / 2)):int(index + (length / 2))]
            if len(extract) != length: continue
            reads.append(extract)
            labs.append(label)
            lengths.append(len(tipds[int(hole)]))
        if subsample != 0:
            mno += 1
            if mno == subsample: break
    new_mat = np.vstack(reads)
    return (new_mat, np.array(lengths), np.array(labs))

def distill_tipds_tad(tipds, valid_zmws, sitelist, dinuclist, length, label, rep_label, subsample=0, min_len=500):
    '''Used to extract relevant mapped ZMWs given a list of ZMW ids. Also excludes bad ZMWs given the valid
    ZMW list. This is for fragment ends (doesn't search for a flat feaure of interest). We also need to validate 
    that findings on the 3 prime end of fragments.'''
    sites = open(sitelist,'r')
    dinucs = open(dinuclist).readlines()
    hole_nos = {}
    dinuc_content = {}
    idx=0
    for i in sites:
        split = i.split()
        nucs = np.array(dinucs[idx].split()[7:],dtype=int)
        idx+=1
        if int(split[0]) not in valid_zmws: continue
        dinuc_content[split[0]] = nucs
        hole_nos[split[0]] = [split[1], int(split[3]) - int(split[2]), split[4]]
    sites.close()
    new_mat = np.zeros(length)
    lengths = []
    labs = []
    reads_f = []
    reads_r = []
    n_mol = 0
    holes = []
    dinucs = []
    for hole in hole_nos:
        if subsample != 0:
            if n_mol >= subsample: break
        read = tipds[int(hole)]
        read_r = read[::-1]
        dist = hole_nos[hole]
        dinuc_vec = dinuc_content[hole]
        if len(tipds[int(hole)]) < min_len: continue
        if len(tipds[int(hole)]) >= length:
            reads_f.append(read[:length])
            reads_r.append(read_r[:length])
            lengths.append(len(tipds[int(hole)]))
            labs.append(dist[-1])
            holes.append(valid_zmws[int(hole)])
            dinucs.append(dinuc_vec)
            n_mol +=1
        elif len(tipds[int(hole)]) < length:
            filler = np.empty(length - len(tipds[int(hole)]))
            filler[:] = np.nan
            lengths.append(len(read))
            read = np.append(read, filler)
            read_r = np.append(read_r, filler)
            reads_f.append(read)
            reads_r.append(read_r)
            labs.append(dist[-1])
            holes.append(valid_zmws[int(hole)])
            dinucs.append(dinuc_vec)
            n_mol +=1
    new_mat = np.vstack(reads_f)
    new_mat_r = np.vstack(reads_r)
    dinuc_mat = np.vstack(dinucs)
    return (new_mat, new_mat_r, lengths, labs,holes,dinucs)

def process_xcors(new_mat, max_len=1000, lengths=False):
    process = np.nan_to_num(pd.DataFrame(new_mat).rolling(33,axis=1, center=True, min_periods=1).mean())
    if isinstance(lengths, np.ndarray):
        auto_cor = xcor(process, process, maxlen=max_len, lengths=lengths)
    else:
        auto_cor = xcor(process, process, maxlen=max_len)
    process = None
    return auto_cor

def cluster_mats(matrix, res=0.8,neighbors=15):
    mat = scanpy.AnnData(X=np.nan_to_num(matrix))
    scanpy.tl.pca(mat)
    scanpy.pp.neighbors(mat,metric='correlation',n_neighbors=neighbors)
    scanpy.tl.leiden(mat,resolution=res)
    return (np.array(mat.obs['leiden']))

def plotter(new_mat, auto_cor, clusters,smooth):
    for i in np.unique(clusters):
        print(i)
        mat_new = auto_cor[clusters == i]
        print(len(mat_new))
        mat_new2 = new_mat[clusters == i]
        mat_new2 = pd.DataFrame(mat_new2).rolling(smooth, axis=1, center=True, min_periods=1).mean()
        plt.figure(figsize=(30,10))
        subplot(121)
        plt.plot(range(len(mat_new[0])), np.nanmean(mat_new, axis=0))
        xlabel('Offset (bp)')
        ylabel('Pearson\'s r')
        subplot(122)
        plt.plot(range(len(mat_new2[0])), np.nanmean(mat_new2, axis=0))
        xlabel('Distance to 5\'-MNase cut')
        ylabel('Average smoothed IPD (frames)')
        plt.show()

def binarize_sig(mat,cutoff=0.5):
    test = np.copy(mat)
    test[test < cutoff] = 0
    test[test >= cutoff] = 1
    return test

def vec2ggplot(vecs, fho, label):
    for i in range(len(vecs)):
        print("%s\t%s\t%s" % (i, vecs[i], label), file=fho)

## Load up files

This cell loads up the *in vivo* data from the pickle format generated by extractIPD.

In [ ]:
length = 1000
# #BINARIZED
k562_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_chromatin_bingmm'
k562_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_chromatin_rep2_bingmm'
k562_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_chromatin_bingmm'
k562_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_chromatin_rep2_bingmm'

k562_rep1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_chromatin_onlyT'
k562_rep2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_chromatin_rep2_onlyT'
k562_rep3_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_chromatin_onlyT'
k562_rep4_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_chromatin_rep2_onlyT'

k562_neg_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_neg_bingmm'
k562_neg_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_neg_rep2_bingmm'
k562_pos_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_pos_bingmm'
k562_pos_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_pos_rep2_bingmm'

k562_neg_rep1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_neg_onlyT'
k562_neg_rep2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_neg_rep2_onlyT'
k562_pos_rep1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_pos_onlyT'
k562_pos_rep2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_pos_rep2_onlyT'


# neg_rep1 = eat_pickle_binary("%s.pickle" % k562_neg_rep1,  'K562_neg_Rep1')
# neg_rep2 = eat_pickle_binary("%s.pickle" % k562_neg_rep2,  'K562_neg_Rep2')
# pos_rep1 = eat_pickle_binary("%s.pickle" % k562_pos_rep1,  'K562_pos_Rep1')
# pos_rep2 = eat_pickle_binary("%s.pickle" % k562_pos_rep2,  'K562_pos_Rep2')
# neg_rep1 = eat_pickle_binary("%s.pickle" % k562_neg_rep1,  'K562_neg_Rep1')
# neg_rep2 = eat_pickle_binary("%s.pickle" % k562_neg_rep2,  'K562_neg_Rep2')
# pos_rep1 = eat_pickle_binary("%s.pickle" % k562_pos_rep1,  'K562_pos_Rep1')
# pos_rep2 = eat_pickle_binary("%s.pickle" % k562_pos_rep2,  'K562_pos_Rep2')


#NONBINARIZED
# k562_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_chromatin_onlyT'
# k562_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_chromatin_rep2_onlyT'
# k562_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_chromatin_onlyT'
# k562_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_chromatin_rep2_onlyT'

# k562_neg_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_neg_onlyT'
# k562_neg_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_neg_rep2_onlyT'
# # k562_neg_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_neg_onlyT'
# # k562_neg_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_neg_rep2_onlyT'

# rep1, rep1_valid = eat_pickle("%s.pickle" % k562_rep1, "%s_zmwinfo.pickle" % k562_rep1)
# rep2, rep2_valid = eat_pickle("%s.pickle" % k562_rep2, "%s_zmwinfo.pickle" % k562_rep2)
# rep3, rep3_valid = eat_pickle("%s.pickle" % k562_rep3, "%s_zmwinfo.pickle" % k562_rep3)
# rep4, rep4_valid = eat_pickle("%s.pickle" % k562_rep4, "%s_zmwinfo.pickle" % k562_rep4)
# neg_rep1, rep3_valid = eat_pickle("%s.pickle" % k562_neg_rep1, "%s_zmwinfo.pickle" % k562_rep3)
# neg_rep2, rep4_valid = eat_pickle("%s.pickle" % k562_neg_rep2, "%s_zmwinfo.pickle" % k562_rep4)

## Load all files (direct to array)
A version of the eat_pickle function that loads the data directly to an array instead of a dictionary. We include here two length cutoffs -- we ignore all molecules < 500 nt in length, and also only inspect the first 1000 bp of molecules from the 5' end of the unaligned CCS read.

In [ ]:
length = 1000
rep1, l1, lb1, h1 = eat_pickle_b2m("%s.pickle" % k562_rep1, 'K562_Rep1', length)
rep1, l2, lb2, h2 = eat_pickle_b2m("%s.pickle" % k562_rep2, 'K562_Rep2', length)
rep1, l3, lb3, h3 = eat_pickle_b2m("%s.pickle" % k562_rep3, 'K562_Rep3', length)
rep1, l4, lb4, h4 = eat_pickle_b2m("%s.pickle" % k562_rep4, 'K562_Rep4', length)
#mat_total=np.vstack((rep1,rep2,rep3,rep4))
#lengths_total = np.array(np.concatenate((l1,l2,l3,l4), axis=None))
#lb_total = np.array(np.concatenate((lb1,lb2,lb3,lb4), axis=None))
h_total = np.array(np.concatenate((h1,h2,h3,h4), axis=None))
rep1, rep2, rep3, rep4 = [None, None, None, None]

This cell generates a hash table that connects ZMW hole numbers across multiple runs with their respective indices in the total array (useful for the Figure 5 subsetting analyses).

In [ ]:
zmw2index = {}
idx = 0
for i in range(len(h_total)):
    zmw2index[h_total[i]] = idx
    idx += 1

## Save the array to get around some of these memory leak issues
In case memory is limiting, can save the length filtered IPD arrays to disk.

In [ ]:
np.save('length_filtered_molecules', mat_total)
np.save('length_filtered_lengths',lengths_total)

## Extra Analysis: Deciding on a cutoff to filter out unmethylated molecules
We need to figure out a reasonable mean probability cutoff for each PacBio sequencing run to
filter out molecules that don't seem to harbor much methylation with respect to negative controls. Let's visualize the distributions for each replicate and draw cutoffs from that. It's difficult to see an obvious cutoff in the data (despite it being clear that chromatin & controls are separated), so let's not draw any cutoffs at the moment, especially because we are interested in heterochromatin.

In [ ]:
length = 1000

# mp1 = return_probs("%s.pickle" % k562_rep1)
# mp2 = return_probs("%s.pickle" % k562_rep2)
# mp3 = return_probs("%s.pickle" % k562_rep3)
# mp4 = return_probs("%s.pickle" % k562_rep4)
# nmp1 = return_probs("%s.pickle" % k562_neg_rep1)
# nmp2 = return_probs("%s.pickle" % k562_neg_rep2)
# pmp1 = return_probs("%s.pickle" % k562_pos_rep1)
# pmp2 = return_probs("%s.pickle" % k562_pos_rep2)

# mat_rep1, mat_rep1_r, lengths1, mp1 = distill_tipds_tot(rep1, rep1_valid, length, min_len = 0, take_all = True)
# mat_rep2, mat_rep2_r, lengths2, mp2 = distill_tipds_tot(rep2, rep1_valid, length, min_len = 0, take_all = True)
# mat_rep3, mat_rep3_r, lengths3, mp3 = distill_tipds_tot(rep3, rep1_valid, length, min_len = 0, take_all = True)
# mat_rep4, mat_rep4_r, lengths4, mp4 = distill_tipds_tot(rep4, rep1_valid, length, min_len = 0, take_all = True)
# mat_neg_rep1, mat_neg_rep1_r, neg_lengths1, nmp1 = distill_tipds_tot(neg_rep1, rep1_valid, length, min_len = 0, take_all = True) 
# mat_neg_rep2, mat_neg_rep2_r, neg_lengths2, nmp2 = distill_tipds_tot(neg_rep2, rep1_valid, length, min_len = 0, take_all = True)
# mat_pos_rep1, mat_pos_rep1_r, pos_lengths1, pmp1 = distill_tipds_tot(neg_rep1, rep1_valid, length, min_len = 0, take_all = True) 
# mat_pos_rep2, mat_pos_rep2_r, pos_lengths2, pmp2 = distill_tipds_tot(neg_rep2, rep1_valid, length, min_len = 0, take_all = True)

print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % (np.median(mp1),np.median(mp2),np.median(mp3),np.median(mp4),np.median(nmp1),np.median(nmp2),np.median(pmp1),np.median(pmp2)))

plt.figure(figsize=(8,8))
sns.distplot(mp1)
sns.distplot(nmp1)
sns.distplot(pmp1)
plt.show()

plt.figure(figsize=(8,8))
sns.distplot(mp2)
sns.distplot(nmp2)
sns.distplot(pmp2)
plt.show()

plt.figure(figsize=(8,8))
sns.distplot(mp3)
sns.distplot(nmp1)
sns.distplot(pmp1)
plt.show()

plt.figure(figsize=(8,8))
sns.distplot(mp4)
sns.distplot(nmp2)
sns.distplot(pmp2)
plt.show()


## Figure 2: Averages of the modification signal across the first 1 kb of K562 oligonucleosomes.
Here we're sampling molecules to relate the average methylation signature on 1kb of template DNA. We load up the raw data (with length filtration), concatenate all of the resulting matrices from each of the four separate samples / runs, and then plot the NaN-sensitive mean over the matrix as a function of distance along the molecule. This should have the effect of revealing average nucleosome positions, as the termini of each sequenced molecule results from an MNase cut.

In [ ]:
length = 1000
mat_rep1, lengths1 = distill_tipds_tot(rep1, length, min_len = 500)
mat_rep2, lengths2 = distill_tipds_tot(rep2, length, min_len = 500)
mat_rep3, lengths3 = distill_tipds_tot(rep3, length, min_len = 500)
mat_rep4, lengths4 = distill_tipds_tot(rep4, length, min_len = 500)
mat_total = np.vstack((mat_rep1, mat_rep2, mat_rep3, mat_rep4))
mat_rep1,mat_rep2,mat_rep3,mat_rep4 = [None, None, None, None]
lengths_total = np.concatenate((lengths1,lengths2, lengths3, lengths4), axis=None)
lengths1,lengths2,lengths3,lengths4 = [None,None,None,None]
rep1, rep2, rep3, rep4 = [None,None,None,None]
#Controls for plotting the below cell
# mat_neg_rep1, neg_lengths1 = distill_tipds_tot(neg_rep1, length, min_len = 500)
# mat_neg_rep2, neg_lengths2 = distill_tipds_tot(neg_rep2, length, min_len = 500)
# mat_neg_rep1 = np.vstack((mat_neg_rep1,mat_neg_rep2))
# neg_lengths1 = np.append(neg_lengths1,neg_lengths2)
# mat_neg_rep2 = 0
# neg_lengths2 = 0
# mat_pos_rep1, pos_lengths1 = distill_tipds_tot(pos_rep1, length, min_len = 500) 
# mat_pos_rep2, pos_lengths2 = distill_tipds_tot(pos_rep2, length, min_len = 500)
# mat_pos_rep1 = np.vstack((mat_pos_rep1,mat_pos_rep2))
# pos_lengths1 = np.append(pos_lengths1,pos_lengths2)
# mat_pos_rep2 = 0
# pos_lengths2 = 0


Here we're just creating a text file to easily plot the results in ggplot

In [ ]:
# plt.figure(figsize=(20,20))
# plt.plot(range(len(mat_rep1[0])), np.nanmean(mat_rep1,axis=0))
# plt.plot(range(len(mat_rep1[0])), np.nanmean(mat_rep2,axis=0))
# plt.plot(range(len(mat_rep1[0])), np.nanmean(mat_neg_rep1,axis=0))
# plt.plot(range(len(mat_rep1[0])), np.nanmean(mat_pos_rep1,axis=0))
# plt.show()

fho = open('Final_ggplot_Fig2C.txt','w')
vec2ggplot(np.nanmean(mat_rep1,axis=0), fho, 'Replicate 1')
vec2ggplot(np.nanmean(mat_rep2,axis=0), fho, 'Replicate 2')
vec2ggplot(np.nanmean(mat_neg_rep1,axis=0), fho, '(-) Control')
vec2ggplot(np.nanmean(mat_pos_rep1,axis=0), fho, '(+) Control')
fho.close()

## Supplementary Figure 6: ChromHMM Coverage Enrichment Analysis
We downloaded K562 ChromHMM labels from the UCSC Genome Browser, lifted over coordinates to hg38 using the  ```liftOver``` tool, and then used ```bedtools multicov``` to compute the read coverage for each ChromHMM BED entry. We then read this file in as a pandas dataframe. 

In [ ]:
hc_file = pd.read_table('cov_check.bed',header=None)
hc_file.columns = ['chrid','start','end','chromhmm_label','c2','c3','c4','c5','c6','r1_rep1','r1_rep2','r2_rep1','r2_rep2','mnase','dnase','shotgun']

In [ ]:
hc_file

We next use this dataframe to estimate the relative enrichment / depletion of each label in each dataset. The datasets we use in addition to our in-house SAMOSA data, we use aligned ENCODE DNaseI-seq data (accession: ENCFF156LGK), in-house aligned K562 MNase-seq data (PMID: 27151365), and publicly released whole-genome shotgun sequencing CCS data from PacBio (https://downloads.pacbcloud.com/public/dataset/HG002_SV_and_SNV_CCS/consensusreads/). We estimated enrichment by evaluating normalizing the number of reads mapping to each ChromHMM labeled genomic bin, dividing this value by the total number of bins with that ChromHMM label, and taking the natural log. We then plotted this data in heatmap form using ggplot2. 

In [ ]:
tot_cov_expt = np.sum(hc_file['r1_rep1']) +\
    np.sum(hc_file['r1_rep2']) + np.sum(hc_file['r2_rep1']) + np.sum(hc_file['r2_rep2'])
print(tot_cov_expt)
tot_cov_mnase = np.sum(hc_file['mnase'])
print(tot_cov_mnase)
tot_cov_dnase = np.sum(hc_file['dnase'])
print(tot_cov_dnase)
tot_cov_shotgun = np.sum(hc_file['shotgun'])
print(tot_cov_shotgun)
fho = open('supp_figure_enrichments.txt','w')
for i in np.unique(hc_file['chromhmm_label']):
    if i[:3] == 'Gen':continue
    print(i)
    cluster_sub = hc_file[hc_file['chromhmm_label'] == i]
    cluster_frac = len(cluster_sub) / len(hc_file)
    cluster_cov_expt = np.sum(cluster_sub['r1_rep1']) +\
        np.sum(cluster_sub['r1_rep2']) + np.sum(cluster_sub['r2_rep1']) + np.sum(cluster_sub['r2_rep2'])
    cluster_cov_mnase = np.sum(cluster_sub['mnase'])
    cluster_cov_dnase = np.sum(cluster_sub['dnase'])
    cluster_cov_shotgun = np.sum(cluster_sub['shotgun'])
    cluster_log_odds = np.log(cluster_cov_expt / tot_cov_expt / cluster_frac)
    mnase_log_odds = np.log(cluster_cov_mnase / tot_cov_mnase / cluster_frac)
    dnase_log_odds = np.log(cluster_cov_dnase / tot_cov_dnase / cluster_frac)
    shotgun_log_odds = np.log(cluster_cov_shotgun / tot_cov_shotgun / cluster_frac)
    odds = [cluster_log_odds, mnase_log_odds, dnase_log_odds, shotgun_log_odds]
    labels = ['SAMOSA','MNase-seq','DNase-seq','Shotgun CCS']
    for j in range(len(odds)):
        print("%s\t%s\t%s" % (i,odds[j],labels[j]), file = fho)
fho.close()

## Figure 3: Clustering analysis of all chromatin molecules >=500 bp in length
We used Leiden clustering (PMID: 30914743) to take all molecules in our dataset passing our lower length cutoff and subjected them to unbiased clustering. Resolution and n_neighbors were manually adjusted to avoid generating large numbers of very small clusters (i.e. < 100 molecules). All parameters used for plotting figures in the paper are recapitulated in the Jupyter notebook. Our clustering strategy was as follows: first, we smoothed raw signal matrices with a 33 bp NaN-sensitive running mean. We next computed the autocorrelation function for each molecule in the matrix, using the full length of the molecule up to 1000 bp. We then used Scanpy (PMID: 29409532) to perform Leiden clustering on the resulting matrix. We visualized the resulting cluster averages with respect to the average autocorrelation function, and with respect to averaged modification probabilities for each cluster. For a subset of clusters we also randomly sampled n molecules to directly visualize in the paper.

In [ ]:
auto_cor = np.load('length_filtered_auto_cor.npy')
np.nan_to_num(auto_cor, copy=False)
mat = scanpy.AnnData(X=auto_cor[:,:500])
scanpy.tl.pca(mat)
scanpy.pp.neighbors(mat,metric='correlation',n_neighbors=10)
scanpy.tl.leiden(mat,resolution=0.4)
clusters = np.array(mat.obs['leiden'])

## Save cluster labels to text file
We saved the cluster labels to a text file for later access, as clustering took > 12 hours on our server.

In [ ]:
fho = open('final_cluster_labels_total.txt','w')
for i in range(len(clusters)):
    print("%s\t%s" % (i, clusters[i]), file = fho)

fho.close()

## Figure 3: Inspect clusters, compute single-molecule autocorrelograms.
We computed single-molecule autocorrelograms and discovered peaks on these autocorrelograms as follows: for each molecule, we used the scipy ```find_peaks``` function to in the computed autocorrelogram and annotated the location of that peak. We also kept track of the molecules where ```find_peaks``` could not detect a peak using the given parameters (optimized manually by modifying peak height / width to detect peaks on the averaged autocorrelograms; detection of a peaks between 180-190 bp, the expected NRL in human cells suggests that these parameters are sound). For each collection of single-molecule autocorrelogram peaks we computed the median, the median absolute deviation, and visualized the distribution of peak locations as a histogram.

In [ ]:
def peak_cruncher(ac):
    peaks1 = []
    for i in range(len(ac)):
        mol1 = ac[i]
        peak1, data = signal.find_peaks(mol1, height = 0.10, width = 25)
        if len(peak1) == 0:
            ac1p = np.nan
        else:
            ac1p = int(peak1[0])
        peaks1.append(ac1p)
    peaks1 = np.array(peaks1)
    med1 = np.nanmedian(peaks1)
    mad1 = scipy.stats.median_absolute_deviation(peaks1,nan_policy='omit')
    mis1 = np.count_nonzero(np.isnan(peaks1))
    return (peaks1, med1, mad1,mis1)

cluster_file = open('final_cluster_labels_total.txt')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)
print(clusters)
smooth = 33
print("this happens")
fho1 = open('Fig3_hmaps.txt','w')
fho2 = open('Fig3_autocors.txt','w')
fho3 = open('Fig3_distros.txt','w')
fho4 = open('Fig3_sample_hmaps.txt', 'w')
for i in np.unique(clusters):
    if i == '7': continue
    mat_total = np.load('length_filtered_molecules.npy')[clusters == i]
    mat_total = np.nanmean(pd.DataFrame(mat_total).rolling(smooth, axis=1, center=True, min_periods=1).mean(),axis=0)
    auto_cor = np.load('length_filtered_auto_cor.npy')[clusters == i]
    peaks1, med1, mad1, mis1 = peak_cruncher(auto_cor)
    overall_peak, data = signal.find_peaks(np.nanmean(auto_cor,axis=0), height = 0.10, width = 25) 
    auto_cor = np.nanmean(auto_cor,axis=0)
    for j in range(len(mat_total)):
        print("%s\t%s\t%s" % (i, j, mat_total[j]), file=fho1)
    for j in range(len(auto_cor)):
        print("%s\t%s\t%s" % (i, j, auto_cor[j]), file=fho2)
    print(len(mat_total))
    print(i)
    for j in range(len(peaks1)):
        print("%s\t%s\t%s\t%s\t%s" % (i, peaks1[j], overall_peak, mis1, len(mat_total)), file=fho3)
    if len(overall_peak) == 0:
        overall_peak = 0
    else:
        overall_peak = overall_peak[0]
    print("%s\t%s\t%s\t%s" % (med1,mad1,mis1,overall_peak))
    plt.figure(figsize=(30,10))
    subplot(131)
    plt.plot(range(1000), auto_cor)
    xlabel('Offset (bp)')
    ylabel('Pearson\'s r')
    subplot(132)
    plt.plot(range(1000), mat_total)
    xlabel('Distance to 5\'-MNase cut')
    ylabel('Average smoothed IPD (frames)')
    subplot(133)
    sns.distplot(peaks1[~np.isnan(peaks1)])
    axvline(overall_peak)
    plt.show()
#     if 
#     for j in range(len(mat_total)):
#         for k in range(len(mat_total[j])):
#             print("%s\t%s\t%s" % (i, j, mat_total[j][k]), file=fho1)        
            
fho1.close()
fho2.close()
fho3.close()
fho4.close()

### Write heatmap files to ggplottable txt file for easy data frame manipulation.
We wrote the resulting signal averages to a text file for easy import to R or Pandas.

In [ ]:
def peak_cruncher(ac):
    peaks1 = []
    for i in range(len(ac)):
        mol1 = ac[i]
        peak1, data = signal.find_peaks(mol1, height = 0.10, width = 25)
        if len(peak1) == 0:
            ac1p = np.nan
        else:
            ac1p = int(peak1[0])
        peaks1.append(ac1p)
    peaks1 = np.array(peaks1)
    med1 = np.nanmedian(peaks1)
    mad1 = scipy.stats.median_absolute_deviation(peaks1,nan_policy='omit')
    mis1 = np.count_nonzero(np.isnan(peaks1))
    return (peaks1, med1, mad1,mis1)

cluster_file = open('final_cluster_labels_total.txt')
lengths_total = np.load('length_filtered_lengths.npy')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)
print(clusters)
smooth = 33
print("this happens")

fho4 = open('Fig3_sample_hmaps.txt', 'w')
for i in np.unique(clusters):
    if i == '0' or i == '2' or i == '6':
        mat_total = np.load('length_filtered_molecules.npy')[(clusters == i) & (lengths_total >= 1000)]
        mat_total = pd.DataFrame(mat_total).rolling(smooth, axis=1, center=True, min_periods=1).mean().values
        print(len(mat_total))
        idx = np.random.choice(mat_total.shape[0], 5000, replace=False)
        mat_total = mat_total[idx,:]
        for j in range(len(mat_total)):
            for k in range(len(mat_total[j])):
                print("%s\t%s\t%s\t%s" % (i, j, k, mat_total[j][k]), file=fho4)
fho4.close()

In [ ]:
print(mat_total.shape[0])

## Figure 3: Cluster Statistics
We'd like to define the 1.) average modification probability, 2.) cluster percentage, and 3.) cluster NRL distributions which we visualize as a violin plot, stacked bar plot, and violin plot, respectively. To aid in visualizing all of these easily, we write all of these values to a single txt file that can be loaded into R or Pandas.   

In [ ]:
print(len(lb_total))

In [ ]:
mat_total = np.load('length_filtered_molecules.npy')
cluster_file = open('final_cluster_labels_total.txt')
lengths_total = np.load('length_filtered_lengths.npy')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)
mat_total = np.nanmean(mat_total, axis=1)
fho = open('Fig3BCD.txt','w')
for i in range(len(clusters)):
    if clusters[i] == '7': continue
    print("%s\t%s\t%s\t%s" % (clusters[i], lengths_total[i], lb_total[i], mat_total[i]), file=fho)
fho.close()

## Extra Analysis: Fragment length fold enrichment. 
This analysis did not make it into the paper, but we also computed the relative enrichment / depletion of different fragment lengths as in Risca *et al* (PMID: 28024297) and Ramani *et al* (PMID: 30811994) to determine what fragment length classes were favored across each cluster.

In [ ]:
frame = pd.read_table('Fig3BCD.txt')
frame.columns = ['clust_label','length','replicate','methyl_mean']
bg_fld = frame[frame['length'] <= 2000]['length'].value_counts() / len(frame[frame['length'] <= 2000]['length'])
bg_fld = dict(zip(bg_fld.index.values, bg_fld.values))
tot_mat = []
fho = open('FLFE_Fig4.txt','w')
for i in np.unique(frame['clust_label']):
    clust_vals = []
    sub_clust = frame[(frame['length'] <= 2000) & (frame['clust_label'] == i)]['length']
    clust_fld = sub_clust.value_counts() / len(sub_clust)
    clust_fld = dict(zip(clust_fld.index.values, clust_fld.values))
    for j in range(500,2000):
        if j in clust_fld:
            val = np.log2(clust_fld[j] / bg_fld[j])
            clust_vals.append(np.log2(clust_fld[j] / bg_fld[j]))
        else:
            val = np.nan
        print("%s\t%s\t%s" % (i, j, val), file=fho)
    tot_mat.append(clust_vals)
fho.close()
# tot_mat = np.vstack(tot_mat)
# plt.figure(figsize=(20,20))
# imshow(tot_mat, cmap=cm.RdBu_r,aspect=100)
# colorbar()
# plt.show()

In [ ]:
#Cluster fractions
for i in np.unique(frame['clust_label']):
    print(i)
    print(len(frame[(frame['clust_label'] == i)]['clust_label']) / len(frame['clust_label']))

## Figure 4: Transcription factor binding motif analyses.
Transcription factor binding sites were obtained as in Ramani et al. Briefly, we downloaded IDR-filtered ENCODE ChIP-seq peaks for CTCF, NRF1, REST, c-MYC, PU.1, and GATA1, and then used FIMO to predict TF binding sites within these peaks using CISTROME PWM definitions for each transcription factor. For MNase-cleavage analyses, we plotted the abundance of MNase cuts (2 per molecule) with respect to TF binding sites and plotted these as number of cleavages per molecules sequenced.


To examine modification probabilities around TF binding sites, we wrote a custom script (```zmw_selector.py```) to find the ZMWs that overlap with features of interest (e.g. transcription factor binding sites). We extracted all ZMWs where a portion of the read alignment falls within 1 kb of a given feature, and annotated the position of the alignment starts, ends, and strand with respect to the feature. We then used these coordinates and strand information to extract all modification signal falling within a 500 bp window centered at each TF binding site.

## Read in data as dict

In [ ]:
k562_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_chromatin_bingmm'
k562_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_chromatin_rep2_bingmm'
k562_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_chromatin_bingmm'
k562_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_chromatin_rep2_bingmm'

rep1 = eat_pickle_binary("%s.pickle" % k562_rep1,  'K562_Rep1')
rep2 = eat_pickle_binary("%s.pickle" % k562_rep2,  'K562_neg_Rep2')
rep3 = eat_pickle_binary("%s.pickle" % k562_rep3,  'K562_pos_Rep1')
rep4 = eat_pickle_binary("%s.pickle" % k562_rep4,  'K562_pos_Rep2')


## MNase-cleavage analysis around transcription factor binding sites

In [ ]:
tfs = open('./pbrun4_purple/biology_analyses/tfs.txt')
#tfs = ['CTCF\n']
tot_zmws = 247302 + 226280 + 732496 + 747401
for line in tfs:
    tf = "%s_flat" % (line.split()[0])
    test_file1 = open('./pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    test_file2 = open('./pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    test_file3 = open('./pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    test_file4 = open('./pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    files = [test_file1,test_file2,test_file3,test_file4]
    #files = [test_file1,test_file2]
    length = 1000
    meta = np.zeros(length)
    for test_file in files:
        for line in test_file:
            split = line.split()
            start = int(split[2])
            end = int(split[3])
            frag = int(split[4])
            if split[-1] == '+':
                dist1 = start - frag + int(length / 2)
                dist2 = end - frag + int(length / 2)
                if 0 <= dist1 < length:
                    meta[dist1] += 1
                if 0 <= dist2 < length:
                    meta[dist2] += 1
            else:
                dist1 = frag - start + int(length / 2)
                dist2 = frag - end + int(length / 2)
                if 0 <= dist1 < length:
                    meta[dist1] += 1
                if 0 <= dist2 < length:
                    meta[dist2] += 1
    meta=pd.Series(meta).rolling(5,center=True, min_periods=1).mean()
    plt.figure(figsize=(10,10))
    plt.plot(range(len(meta)), meta)
    plt.show()
    for test_file in files:
        test_file.close()
    fho = open('%s_MNase.txt' % tf,'w')
    for i in range(len(meta)):
        print("%s\t%s\t%s" % (i, meta[i] / tot_zmws * 1000000, tf), file=fho)
    fho.close()


## Modification probability analysis around six transcription factors

In [ ]:
length = 500

tfs = open('./pbrun4_purple/biology_analyses/tfs.txt')
#tfs = ['GATA1\n']
lengths_total = np.array([0])
labels_total = np.array([0])
mat_total = []

for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    cntrl_sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    mat_rep1, lengths, labels = distill_tipds_flat_centered(rep1, sites_rep1, length, sample)
    
    cntrl_mat_rep1, cntrl_lengths, cntrl_labels = distill_tipds_flat_centered(rep1, cntrl_sites_rep1, length, "%s_control" % sample ,subsample=len(lengths))    
    
    mat_rep2, lengths2, labels2 = distill_tipds_flat_centered(rep2,  sites_rep2, length, sample)
    
    cntrl_mat_rep2, cntrl_lengths2, cntrl_labels2 = distill_tipds_flat_centered(rep2,  cntrl_sites_rep2, length, "%s_control" % sample,subsample=len(lengths2))    
    
    mat_rep3, lengths3, labels3 = distill_tipds_flat_centered(rep3,  sites_rep3, length, sample)
    
    cntrl_mat_rep3, cntrl_lengths3, cntrl_labels3 = distill_tipds_flat_centered(rep3,cntrl_sites_rep3, length, "%s_control" % sample,subsample=len(lengths3))    
    
    mat_rep4, lengths4, labels4 = distill_tipds_flat_centered(rep4, sites_rep4, length, sample)
    
    cntrl_mat_rep4, cntrl_lengths4, cntrl_labels4 = distill_tipds_flat_centered(rep4, cntrl_sites_rep4, length, "%s_control" % sample,subsample=len(lengths4))    
    
    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4,cntrl_mat_rep1,cntrl_mat_rep2,cntrl_mat_rep3,cntrl_mat_rep4))
#    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4))
    mat_msix = np.vstack((mat_rep1, mat_rep2, mat_rep3, mat_rep4))
    mat_msix_control = np.vstack((cntrl_mat_rep1, cntrl_mat_rep2, cntrl_mat_rep3, cntrl_mat_rep4))
    print("%s\t%s" % (sample, (len(mat_rep1)+len(mat_rep2)+len(mat_rep3)+len(mat_rep4))))
    print("%s_control\t%s" % (sample, (len(cntrl_mat_rep1)+len(cntrl_mat_rep2)+len(cntrl_mat_rep3)+len(cntrl_mat_rep4))))
    lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4, cntrl_lengths, cntrl_lengths2,cntrl_lengths3,cntrl_lengths4))
    labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4,cntrl_labels, cntrl_labels2, cntrl_labels3, cntrl_labels4))
#     lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4))
#     labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4))
    process = pd.Series(np.nanmean(mat_msix, axis=0)).rolling(33,center=True,min_periods=1).mean()
    process_neg = pd.Series(np.nanmean(mat_msix_control, axis=0)).rolling(33,center=True,min_periods=1).mean()
#     process = np.nanmean(binarize_sig(pd.DataFrame(mat_msix).rolling(10, axis=1, center=True, min_periods=1).mean()), axis=0)
#     process_neg = np.nanmean(binarize_sig(pd.DataFrame(mat_msix_control).rolling(10, axis=1, center=True, min_periods=1).mean()), axis=0)
    plt.figure(figsize=(10,10))
    subplot(121)
    plt.plot(range(len(process)), process)
    subplot(122)
    plt.plot(range(len(process_neg)), process_neg)
    plt.show()
    fho=open('%s_m6A.txt' % tf,'w')
    for i in range(len(process)):
        print("%s\t%s\t%s\tsite" % (i, process[i], tf),file=fho)
        print("%s\t%s\t%s\tcontrol" % (i, process_neg[i], tf),file=fho)
    fho.close()

mat_total = np.vstack(mat_total)
lengths_total = lengths_total[1:]
labels_total = labels_total[1:]

tfs.close()

#np.save('length_filtered_molecules_tfs',mat_total)

## Figure 4: Leiden clustering of molecules with TF binding motifs.
Unlike the prior analysis, we do *not* cluster molecules on the basis of the autocorrelation function, and instead use the smoothed modification probabilities instead. We reason that while in the prior case we are explicitly interested in nucleosome regularity, in this case we do not have a strong prior assumption that nucleosomes will be regular or irregular about TF binding sites.

In [ ]:
auto_cor = np.load('length_filtered_molecules_tfs.npy')
auto_cor = pd.DataFrame(auto_cor).rolling(33, axis=1, center=True, min_periods=1).mean()
print(len(auto_cor))
print(len(auto_cor[0]))
mat = scanpy.AnnData(X=np.nan_to_num(auto_cor,copy=False))
scanpy.tl.pca(mat)
scanpy.pp.neighbors(mat,metric='correlation',n_neighbors=10)
scanpy.tl.leiden(mat,resolution=0.7)
clusters = np.array(mat.obs['leiden'])

fho = open('final_cluster_labels_tfs.txt','w')
for i in range(len(clusters)):
    print("%s" % clusters[i], file=fho)
fho.close()

Visualize the resulting clusters and write data to easy-to-plot text file.

In [ ]:
smooth = 33
for i in np.unique(clusters):
    mat_total = np.load('length_filtered_molecules_tfs.npy')[clusters == i]
    auto_cor = np.load('length_filtered_auto_cor_tfs.npy')[clusters == i]
    print(len(mat_total))
    print(i)
    mat_total = pd.DataFrame(mat_total).rolling(smooth, axis=1, center=True, min_periods=1).mean()
    plt.figure(figsize=(30,10))
    subplot(121)
    plt.plot(range(251), np.nanmean(auto_cor, axis=0))
    xlabel('Offset (bp)')
    ylabel('Pearson\'s r')
    subplot(122)
    plt.plot(range(500), np.nanmean(mat_total, axis=0))
    xlabel('Distance to 5\'-MNase cut')
    ylabel('Average smoothed IPD (frames)')
    plt.show()

In [ ]:
cluster_file = open('final_cluster_labels_tfs.txt')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[0])
clusters = np.array(clusters)
print(clusters)
smooth = 33
print("this happens")
fho1 = open('Fig4_hmaps.txt','w')
#fho2 = open('Fig4_autocors.txt','w')
#fho3 = open('Fig3_distros.txt','w')
fho4 = open('Fig4_sample_hmaps.txt', 'w')
for i in np.unique(clusters):
    mat_total = np.load('length_filtered_molecules_tfs.npy')[(clusters == i)]
    mat_total = pd.DataFrame(mat_total).rolling(smooth, axis=1, center=True, min_periods=1).mean().values
    if i == '2' or i == '7' or i == '11':
        print(len(mat_total))
        idx = np.random.choice(mat_total.shape[0], 500, replace=False)
        mat_total = mat_total[idx,:]
        for j in range(len(mat_total)):
            for k in range(len(mat_total[j])):
                print("%s\t%s\t%s\t%s" % (i, j, k, mat_total[j][k]), file=fho4)
#    if i == '7': continue
    mat_total=np.nanmean(mat_total,axis=0)
    for j in range(len(mat_total)):
        print("%s\t%s\t%s" % (i, j, mat_total[j]), file=fho1)
    print(len(mat_total))
    print(i)
   
fho1.close()
fho4.close()

## Supplementary Figure 9: Enrichment Tests for TF Binding
We constructed a series of enrichment tests (Fisher's Exact) to determine odds ratios / p-values to find specific cluster label--transcription factor pairs that were enriched with respect to the total set of all labeled molecules. Finally, we used the Storey q-value package to correct for the number of Fisher's exact tests performed.

In [ ]:
fho = open('fishers_tests_fig3.txt','w')
cluster_df = {'clusters':clusters, 'labels':labels_total}
cluster_df = pd.DataFrame(cluster_df)
for i in np.unique(clusters):
    for j in np.unique(labels_total):
        num_clust_lab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] == j)])
        num_clust_notlab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] != j)])
        num_notclust_lab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] == j)])
        num_notclust_notlab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()
#labs = np.a

In [ ]:
cluster_df.to_csv('tfs_df.csv')

## Figure 5: Enrichment Tests for Chromatin States
We used a custom python script (```zmw_selector_bed.py```) or directly scanned for satellite-containing CCS reads (see Methods section) to extract molecules that fall within ENCODE-defined chromatin states / pertain to human major satellite sequences. We then used the above defined hashmap linking ZMW IDs to indices along the total matrix of molecules to link Cluster IDs and chromatin states. Finally, we constructed a series of enrichment tests (Fisher's Exact) to determine odds ratios / p-values to find specific cluster label--chromatin state pairs that were enriched with respect to the total set of all labeled molecules. Finally, we used the Storey q-value package to correct for the number of Fisher's exact tests performed.

In [ ]:
def find_indices(sites, zmw_dict, samp_label, chrom_label):
    lines = open(sites)
    indices = []
    for entry in lines:
        split = entry.split()
        hole = "%s_%s" % (samp_label, split[0])
        if hole in zmw_dict:
            indices.append(zmw_dict[hole])
    return indices
        
#print(list(zmw2index.keys())[:100])
cluster_file = open('final_cluster_labels_total.txt')
lengths_total = np.load('length_filtered_lengths.npy')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)

#total_dataframe = pd.DataFrame({'cluster_id': [], 'chromatin_type': [], 'total_index': [], 'frag_length' : []})
cluster_ids = []
chromatin_types = []
total_index = []
frag_length = []

tfs = open('./pbrun4_purple/biology_analyses/new_beds.txt')
#tfs = ['coopfoots\n']

for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    print(sample)
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    rep1_indices = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
    rep2_indices = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
    rep3_indices = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
    rep4_indices = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
    tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
    cluster_indices = clusters[tot_indices]
    lengths_indices = lengths_total[tot_indices]
    tot_labels = np.repeat(tf, len(tot_indices))
    cluster_ids.append(cluster_indices)
    chromatin_types.append(tot_labels)
    frag_length.append(lengths_indices)
    total_index.append(tot_indices)

print(cluster_ids)    

#Also include Siva Satellite ZMWs
sites_rep1 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep2 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
sites_rep3 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep4 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
tf='Satellite'
rep1_indices = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
rep2_indices = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
rep3_indices = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
rep4_indices = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
cluster_indices = clusters[tot_indices]
lengths_indices = lengths_total[tot_indices]
tot_labels = np.repeat(tf, len(tot_indices))
cluster_ids.append(cluster_indices)
chromatin_types.append(tot_labels)
frag_length.append(lengths_indices)
total_index.append(tot_indices)

total_dataframe = pd.DataFrame({'cluster_id': np.concatenate(cluster_ids), 'chromatin_type': np.concatenate(chromatin_types) \
    , 'total_index': np.concatenate(total_index), 'frag_length' : np.concatenate(frag_length)})
total_dataframe



In [ ]:
total_dataframe.to_csv('fig5_wholedata.csv')

## Figure 5: Distributions of NRLs
We used the method described above to estimate NRLs on single-molecules and then visualized these distributions, medians and median absolute deviations in Figure 5.

In [ ]:
def peak_cruncher(ac):
    peaks1 = []
    for i in range(len(ac)):
        mol1 = ac[i]
        peak1, data = signal.find_peaks(mol1, height = 0.10, width = 25)
        if len(peak1) == 0:
            ac1p = np.nan
        else:
            ac1p = int(peak1[0])
        peaks1.append(ac1p)
    peaks1 = np.array(peaks1)
    med1 = np.nanmedian(peaks1)
    mad1 = scipy.stats.median_absolute_deviation(peaks1,nan_policy='omit')
    mis1 = np.count_nonzero(np.isnan(peaks1))
    return (peaks1, med1, mad1,mis1)

fho1 = open('fig5_domainmean_plots.txt','w')
fho2 = open('fig5_distplots.txt','w')
smooth = 33
for i in np.unique(total_dataframe['chromatin_type']):
    test_indices = total_dataframe[total_dataframe['chromatin_type'] == i]['total_index']
    mat_total = np.load('length_filtered_molecules.npy')[test_indices]
    auto_cor = np.load('length_filtered_auto_cor.npy')[test_indices]
    peaks1, med1, mad1, mis1 = peak_cruncher(auto_cor)
    overall_peak, data = signal.find_peaks(np.nanmean(auto_cor,axis=0), height = 0.10, width = 25) 
    if len(overall_peak) == 0:
        overall_peak = 0
    else:
        overall_peak = overall_peak[0]
    mat_total= np.nanmean(mat_total,axis=0)
    auto_cor = np.nanmean(auto_cor,axis=0)
    for j in range(len(mat_total)):
        print("%s\t%s\t%s" % (i, j, mat_total[j]), file=fho1)
#     for j in range(len(auto_cor)):
#         print("%s\t%s\t%s" % (i, j, auto_cor[j]), file=fho2)
    print(len(mat_total))
    print(i)
    for j in range(len(peaks1)):
        print("%s\t%s\t%s\t%s\t%s" % (i, peaks1[j], overall_peak, mis1, len(mat_total)), file=fho2)
    print("%s\t%s\t%s\t%s" % (med1,mad1,mis1,overall_peak))
    plt.figure(figsize=(30,10))
    subplot(131)
    plt.plot(range(1000), auto_cor)
    xlabel('Offset (bp)')
    ylabel('Pearson\'s r')
    subplot(132)
    plt.plot(range(1000), mat_total)
    xlabel('Distance to 5\'-MNase cut')
    ylabel('Average smoothed IPD (frames)')
    subplot(133)
    sns.distplot(peaks1[~np.isnan(peaks1)])
    axvline(overall_peak)
    plt.show()

## Extra Analyses: FLFE analyses for different chromatin states.
We did not use these analyses in the paper but we have included these here for those interested in different fragment length enrichments for each chromatin state.

In [ ]:
bg_fld = total_dataframe[total_dataframe['frag_length'] <= 2000]['frag_length'].value_counts() / len(total_dataframe[total_dataframe['frag_length'] <= 2000]['length'])
bg_fld = dict(zip(bg_fld.index.values, bg_fld.values))
tot_mat = []
fho = open('FLFE_Fig4.txt','w')
for i in np.unique(frame['clust_label']):
    clust_vals = []
    sub_clust = frame[(frame['length'] <= 2000) & (frame['clust_label'] == i)]['length']
    clust_fld = sub_clust.value_counts() / len(sub_clust)
    clust_fld = dict(zip(clust_fld.index.values, clust_fld.values))
    for j in range(500,2000):
        if j in clust_fld:
            val = np.log2(clust_fld[j] / bg_fld[j])
            clust_vals.append(np.log2(clust_fld[j] / bg_fld[j]))
        else:
            val = np.nan
        print("%s\t%s\t%s" % (i, j, val), file=fho)
    tot_mat.append(clust_vals)
fho.close()


## Figure 5: Fisher's Exact Tests
Code to perform the Fisher's exact tests described above.

In [ ]:
import scipy as sp
test = total_dataframe[(total_dataframe['chromatin_type'] == 'H3K9me3') | (total_dataframe['chromatin_type'] == 'H3K9me1')]['total_index']
print(len(test))
print(len(np.unique(test)))

fho = open('fishers_tests_fig5.txt','w')
for i in np.unique(total_dataframe['cluster_id']):
    if i == '7': continue
    for j in np.unique(total_dataframe['chromatin_type']):
        num_clust_lab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] == j)])
        num_clust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] != j)])
        num_notclust_lab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] == j)])
        num_notclust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()
                                                                                                                        

# Extra Code
This is extra code we used when troubleshooting a lot of these analyses. Included here for the sake of posterity but not necessary for any analyses in the paper.

## Clustering the TFs and Fisher's tests 

In [ ]:
mat = scanpy.AnnData(X=auto_cor[:,:500])
scanpy.tl.pca(mat)
scanpy.pp.neighbors(mat,metric='correlation',n_neighbors=10)
scanpy.tl.leiden(mat,resolution=0.4)
clusters = np.array(mat.obs['leiden'])
plotter(auto_cor, mat_total, clusters, 33)

# #sample labels for plotting
# sp_obj.obs['samples'] = labels_total
# scanpy.tl.umap(sp_obj)
# #mat_ann.obs['colours'] = color_labels
# fig, ax = plt.subplots(figsize=(20, 14))
# scanpy.pl.umap(sp_obj, color='samples', ax=ax,size=30)
# plt.show()

# labs = np.array(labels_total)

# print(len(labs))
# print(len(clusters))

# #labs = np.array(labels)
# #print("Cluster\t%s\t%s\tOR" % (sample2, sample1))
# for i in np.unique(clusters):
#     for j in np.unique(labs):
#         for k in np.unique(labs):
#             sub = labs[clusters == i]
#             num_s2 = len(sub[sub == j]) 
#             num_not_s2 = len(labs[labs == j]) - num_s2
#             num_s1 = len(sub[sub == k]) 
#             num_not_s1 = len(labs[labs == k]) - num_s1
#             odds_r, pval = sp.stats.fisher_exact([[num_s2, num_s1], [num_not_s2, num_not_s1]])
#             print("%s\t%s\t%s\t%s\t%s\t%s\t%s" % (i, num_s2, num_s1,odds_r,pval, j , k))
    #num_neg2 = len(sub[sub == '%s_neg' % sample2]) / len(labs[labs=='%s_neg' % sample2])
    #print("%s\t%s\t%s\t%s\t%s\t%s" % (i, num_control, num_ctcf, num_neg, num_neg2, lo))
    #print("%s\t%s\t%s\t%s" % (i, num_control, num_ctcf,lo)) 

## Plot cluster accessiblity as a heatmap?

In [ ]:
fho1 = open('hmap_figure.txt','w')
fho2 = open('single_mol_heatmaps.txt', 'w')
for i in np.unique(clusters):
    mat_new = auto_cor[clusters == i]
    mat_new2 = mat_total[clusters == i]
    hmap = pd.Series(np.nanmean(binarize_sig(mat_new2),axis=0)).rolling(20,center=True, min_periods=1).mean()
    for j in range(len(hmap)):
        print("%s\t%s\t%s" % (i, j, hmap[j]), file=fho1)        
    process = pd.DataFrame(binarize_sig(mat_new2)).rolling(5, axis=1, center=True, min_periods=1).mean().values
    for j in range(len(process)):
        for k in range(len(process[j])):
            print("%s\t%s\t%s\t%s\t%s" % (labels_total[j],j , k, process[j][k], i), file=fho2)
    plt.figure(figsize=(5,5))
    plt.plot(range(len(mat_new2[0])), np.nanmean(process, axis=0))
    xlabel('Distance to 5\'-MNase cut')
    ylabel('Average smoothed IPD (frames)')
    subplot(143)
    imshow(mat_new, interpolation=None, cmap=cm.BuPu)
    xlabel('Offset (bp)')
    subplot(144)
    imshow(process, interpolation=None, cmap=cm.BuPu)
    xlabel('Distance')
fho1.close()
fho2.close()

## individual cluster heatmaps

In [ ]:
fho2 = open('single_mol_heatmaps.txt', 'w')
process = pd.DataFrame(binarize_sig(mat_total)).rolling(5, axis=1, center=True, min_periods=1).mean().values
for i in np.unique(clusters):
    labs = labels_total[clusters == i]
    sub = process[clusters == i]
    for j in range(len(sub)):
        for k in range(len(sub[j])):
            print("%s\t%s\t%s\t%s\t%s" % (labs[j],j, k, sub[j][k], i), file=fho2)
fho2.close()


In [ ]:
print(len(process))

## pandas fishers tests for fig 3

In [ ]:
#just do the damn fishers tests in pandas
fho = open('fishers_tests_fig3.txt','w')
cluster_df = {'clusters':clusters, 'labels':labels_total}
cluster_df = pd.DataFrame(cluster_df)
for i in np.unique(clusters):
    for j in np.unique(labels_total):
        num_clust_lab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] == j)])
        num_clust_notlab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] != j)])
        num_notclust_lab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] == j)])
        num_notclust_notlab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()
#labs = np.array(labels)
#print("Cluster\t%s\t%s\tOR" % (sample2, sample1))

In [ ]:
#QuickPlots
fho = open('ggplot_UMAPs_TFs.txt','w')
for i in range(len(labels_total)):
    print("%s\t%s\t%s\t%s"%(sp_obj.obsm['X_umap'][:,0][i], sp_obj.obsm['X_umap'][:,1][i], labels_total[i], clusters[i]), file=fho)
fho.close()


## Examine end-positioned regularity across chromatin domains

In [ ]:
auto_cor, processed = process_xcors(mat_total, length)
print(len(auto_cor))
motif_sites = processed[:,int(length / 2) - 10:int(length / 2) + 10]
clusters = cluster_mats(auto_cor,res=0.3)
plotter(mat_total, auto_cor, clusters)

labs = np.array(labels_total)

print(len(labs))
print(len(np.unique(clusters)))

fho = open('plotme_fisherstests_500.txt','w')

print("Cluster\t%s\t%s\tOR" % (sample2, sample1))
for i in np.unique(clusters):
    sub = labs[clusters == i]
    if len(sub) < 100: continue
    for j in np.unique(labs):
        for k in np.unique(labs):
            num_s2 = len(sub[sub == j]) 
            num_not_s2 = len(labs[labs == j]) - num_s2
            num_s1 = len(sub[sub == k]) 
            num_not_s1 = len(labs[labs == k]) - num_s1
            odds_r, pval = sp.stats.fisher_exact([[num_s2, num_s1], [num_not_s2, num_not_s1]])
            print("%s\t%s\t%s\t%s\t%s\t%s\t%s" % (i, num_s2, num_s1,odds_r,pval, j , k), file=fho)
fho.close() 

In [ ]:
auto_cor, processed = process_xcors(mat_total, length)
print(len(auto_cor))
motif_sites = processed[:,int(length / 2) - 10:int(length / 2) + 10]
clusters = cluster_mats(processed,res=0.5)
plotter(mat_total, auto_cor, clusters)

# labs = np.array(labels_total)

# print(len(labs))
# print(len(clusters))

# fho = open('plotme_fisherstests_750.txt','w')
# print("Cluster\t%s\t%s\tOR" % (sample2, sample1))
# for i in np.unique(clusters):
#     sub = labs[clusters == i]
#     if len(sub) < 100: continue
#     for j in np.unique(labs):
#         for k in np.unique(labs):
#             num_s2 = len(sub[sub == j]) 
#             num_not_s2 = len(labs[labs == j]) - num_s2
#             num_s1 = len(sub[sub == k]) 
#             num_not_s1 = len(labs[labs == k]) - num_s1
#             odds_r, pval = sp.stats.fisher_exact([[num_s2, num_s1], [num_not_s2, num_not_s1]])
#             print("%s\t%s\t%s\t%s\t%s\t%s\t%s" % (i, num_s2, num_s1,odds_r,pval, j , k), file=fho)
# fho.close() 

## 3D Genome Analyses I: Similarity and difference of nucleosome conformers across + within TADs

In [ ]:

length = 1000

tf = 'TAD'
sample = '%s' % tf
sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample

sites_rep1_dinucs = './pbrun4_purple/ccs/for_dinucs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws.extract.bed.facount' % sample
sites_rep2_dinucs = './pbrun4_purple/ccs/for_dinucs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws.extract.bed.facount' % sample
sites_rep3_dinucs = './pbrun6/ccs/for_dinucs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws.extract.bed.facount'% sample
sites_rep4_dinucs = './pbrun6/ccs/for_dinucs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws.extract.bed.facount' % sample

mat_rep1, mat_rep1_r, lengths, labels,holes,dinucs = distill_tipds_tad(rep1, rep1_valid, sites_rep1, sites_rep1_dinucs, length, sample, 'Replicate1', subsample=0)
mat_rep2, mat_rep2_r, lengths2, labels2,holes2,dinucs2 = distill_tipds_tad(rep2, rep2_valid, sites_rep2, sites_rep2_dinucs, length, sample, 'Replicate2', subsample=0)
mat_rep3, mat_rep3_r, lengths3, labels3,holes3,dinucs3 = distill_tipds_tad(rep3, rep3_valid, sites_rep3, sites_rep3_dinucs, length, sample,'Replicate1', subsample=0)
mat_rep4, mat_rep4_r, lengths4, labels4,holes4,dinucs4 = distill_tipds_tad(rep4, rep4_valid, sites_rep4, sites_rep4_dinucs, length, sample,'Replicate2', subsample=0)

mat_total = np.vstack((mat_rep1,mat_rep2,mat_rep3,mat_rep4))
mat_total_r = np.vstack((mat_rep1_r,mat_rep2_r,mat_rep3_r,mat_rep4_r))

dinucs_total = np.vstack((dinucs,dinucs2,dinucs3,dinucs4))
lengths_total = np.concatenate((lengths, lengths2,lengths3,lengths4))
labels_total = np.concatenate((labels, labels2,labels3,labels4))
holes_total = np.concatenate((holes, holes2, holes3, holes4))

mat_rep1 = 0
mat_rep2 = 0
mat_rep3 = 0
mat_rep4 = 0
mat_rep1_r = 0
mat_rep2_r = 0
mat_rep3_r = 0
mat_rep4_r = 0


In [ ]:
holes_total = np.concatenate((holes, holes2, holes3, holes4))


In [ ]:
print(len(holes_total))
fho = open('forMehran_cluster_zmw_ids','w')
for i in range(len(holes_total)):
    print("%s\t%s" % (holes_total[i], clusters[i]),file=fho)
fho.close()
    

## Read in the previously calc'd cluster labels

In [ ]:
cluster_file = open('./fig4_cluster_labs.txt')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[0])
clusters = np.array(clusters)
print(len(clusters))

## Cluster TAD-binned ZMWs by auto-correlation

In [ ]:
def plotter_v3(new_mat, new_mat2, auto_cor, clusters,smooth):
    for i in np.unique(clusters):
        print(i)
        mat_new = auto_cor[clusters == i]
        print(len(mat_new))
        mat_new2 = new_mat[clusters == i]
        process = pd.DataFrame(mat_new2).rolling(smooth, axis=1, center=True, min_periods=1).mean()
        mat_new3 = new_mat2[clusters == i]
        process = pd.DataFrame(mat_new2).rolling(smooth, axis=1, center=True, min_periods=1).mean()
        process2 = pd.DataFrame(mat_new3).rolling(smooth, axis=1, center=True, min_periods=1).mean()
        plt.figure(figsize=(30,10))
        subplot(131)
        plt.plot(range(len(mat_new[0])), np.nanmean(mat_new, axis=0))
        xlabel('Offset (bp)')
        ylabel('Pearson\'s r')
        subplot(132)
        plt.plot(range(len(mat_new2[0])), np.nanmean(process, axis=0))
        plt.plot(range(len(mat_new2[0])), np.nanmean(process2, axis=0))
        xlabel('Distance to 5\'-MNase cut')
        ylabel('Average smoothed IPD (frames)')
        subplot(133)
        imshow(mat_new, interpolation=None, cmap=cm.BuPu)
        xlabel('Offset (bp)')
        plt.show()

nl = 500
auto_cor = 0
processed = 0
auto_cor1, processed = process_xcors(np.nan_to_num(mat_total[:,:nl]), nl)
processed = 0
auto_cor2, processed = process_xcors(np.nan_to_num(mat_total_r[:,:nl]), nl)
processed = 0
#clusters, sp_obj = cluster_mats(auto_cor1,res=0.8,neighbors=15)
#plotter_v3(binarize_sig(mat_total), binarize_sig(mat_total_r), auto_cor, clusters, 5)

## Save cluster labels for later

In [ ]:
fho = open('fig4_cluster_labs_v2.txt','w')

for i in range(len(clusters)):
    print("%s\t%s\t%s" % (clusters[i], labels_total[i], lengths_total[i]), file=fho)

fho.close()
    

## Plot each conformer A/T sig + autocorr


In [ ]:
fho = open('fig4_lineplots.txt','w')
for i in np.unique(clusters):
    cluster_size = len(mat_total[clusters == i])
    if cluster_size < 1000:
        continue
    sub = pd.Series(np.nanmean(binarize_sig(mat_total[(clusters == i)]),axis=0)).rolling(20,center=True, min_periods=1).mean()
    label = i
    for j in range(len(sub)):
        print("%s\t%s\t%s\t%s" % (j, sub[j], label,cluster_size),file=fho)
fho.close()

## Heatmap representations for Fig 4

In [ ]:
fho1 = open('hmap_figure_fig4.txt','w')
for i in np.unique(clusters):
    mat_new = auto_cor[clusters == i]
    if len(mat_new) < 1000: continue
    hmap = np.nanmean(mat_new, axis=0)
    for j in range(len(hmap)):
        print("%s\t%s\t%s" % (i, j, hmap[j]), file=fho1)        
fho1.close()


## Sort the clustered autocorrelograms?

In [ ]:
def peak_cruncher(autocorrelogram):
    peaks = []
    heights = []
    non_peaks = 0
    for i in range(len(autocorrelogram)):
        mol = autocorrelogram[i]
        peak, data = signal.find_peaks(mol, height = 0.10, width = 25)
        if len(peak) == 0:
            non_peaks+=1
            continue
        peaks.append(peak[0])
        heights.append(data['peak_heights'][0])
    #for after I implement filtration via controls
#         else:
#             peaks.append(0)
#             heights.append(0)
    med = np.median(peaks)
    mad = scipy.stats.median_absolute_deviation(peaks)
    return (peaks, heights, non_peaks, med, mad)

def plotter_v2(new_mat, auto_cor, clusters, fho):
    for i in np.unique(clusters):
        mat_new = auto_cor[clusters == i]
        peaks, heights, non_peaks, med, mad = peak_cruncher(mat_new)
        mat_mean = np.mean(mat_new, axis=0)
        overall_peak, data = signal.find_peaks(mat_mean, height = 0.10, width = 25)
        if len(overall_peak) == 0:
            overall_peak = 0
        else:
            overall_peak = overall_peak[0]
        length_sub = lengths_total[clusters == i]
        print("%s\t%s\t%s\t%s\t%s\t%s" % (i, overall_peak, non_peaks, len(length_sub) , med, mad))
        print("%s\t%s\t%s\t%s\t%s\t%s" % (i, overall_peak, non_peaks, len(length_sub), med, mad), file=fho)
#         plt.figure(figsize=(10,10))
#         subplot(121)
#         sns.distplot(peaks)
#         axvline(overall_peak)
#         subplot(122)
#         sns.distplot(length_sub,bins=50)
#         plt.show()
fho = open('figs4_statistics_bothstrands.txt','w')
plotter_v2(mat_total, auto_cor, clusters, fho)
fho.close()

## Compute single-molecule autocorrelograms and find peaks

In [ ]:
def peak_cruncher(ac1,ac2):
    peaks1 = []
    peaks2 = []
    for i in range(len(ac1)):
        mol1 = ac1[i]
        mol2 = ac2[i]
        peak1, data = signal.find_peaks(mol1, height = 0.10, width = 25)
        peak2, data = signal.find_peaks(mol2, height = 0.10, width = 25)
        if len(peak1) == 0:
            ac1p = np.nan
        else:
            ac1p = int(peak1[0])
        if len(peak2) == 0:
            ac2p = np.nan
        else:
            ac2p = int(peak2[0])
        peaks1.append(ac1p)
        peaks2.append(ac2p)
    peaks1 = np.array(peaks1)
    peaks2 = np.array(peaks2)
    med1 = np.nanmedian(peaks1)
    mad1 = scipy.stats.median_absolute_deviation(peaks1,nan_policy='omit')
    mis1 = np.count_nonzero(np.isnan(peaks1))
    med2 = np.nanmedian(peaks2)
    mad2 = scipy.stats.median_absolute_deviation(peaks2,nan_policy='omit')
    mis2 = np.count_nonzero(np.isnan(peaks2))
    return (peaks1, peaks2, med1, mad1,mis1, med2, mad2,mis2)

def smol_autos(auto_cor1, auto_cor2, clusters, fho):
    for i in np.unique(clusters):
        mat_new = auto_cor1[clusters == i]
        mat_new2 = auto_cor2[clusters == i]
        peaks1, peaks2, med1, mad1, mis1, med2, mad2, mis2 = peak_cruncher(mat_new, mat_new2)
        mat_mean = np.mean(mat_new, axis=0)
        mat_mean2 = np.mean(mat_new2, axis=0)
        overall_peak, data = signal.find_peaks(mat_mean, height = 0.10, width = 25)
        overall_peak2, data = signal.find_peaks(mat_mean2, height = 0.10, width = 25)
        if len(overall_peak2) == 0:
            overall_peak2 = 0
        else:
            overall_peak2 = overall_peak2[0]
        if len(overall_peak) == 0:
            overall_peak = 0
        else:
            overall_peak = overall_peak[0]
        length_sub = lengths_total[clusters == i]
        if len(length_sub) <= 1000: continue
        print(len(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))]))
        print(len(peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))]))

        print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % \
              (i, overall_peak, overall_peak2, mis1, mis2, med1, med2, mad1, mad2, len(length_sub)))
        print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % \
              (i, overall_peak, overall_peak2, mis1, mis2, med1, med2, mad1, mad2, len(length_sub)),file=fho
              )
        plt.figure(figsize=(5,5))
        sns.jointplot(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], kind="hex")
        plt.show()
        plt.figure(figsize=(5,5))
        sns.jointplot(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], length_sub[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], kind="hex")        
        plt.figure(figsize=(5,5))
        sns.jointplot(peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], length_sub[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], kind="hex")
#        plt.scatter(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))],alpha=0.25)
        plt.show()
#         plt.figure(figsize=(10,10))
#         subplot(121)
#         sns.distplot(peaks)
#         axvline(overall_peak)
#         subplot(122)
#         sns.distplot(length_sub,bins=50)
#         plt.show()
fho = open('figs4_statistics_bothstrands.txt','w')
smol_autos(auto_cor1, auto_cor2, clusters, fho)
fho.close()

In [ ]:
print(clusters)

## Pick out three clusters to plot in the main text figure; randomly sample 5000 ZMWs

In [ ]:
process=0
fho2 = open('single_mol_heatmaps_fig4.txt', 'w')
#process = pd.DataFrame(binarize_sig(mat_total)).rolling(5, axis=1, center=True, min_periods=1).mean().values
for i in np.unique(clusters):
    idx = np.random.choice(mat_total.shape[0], 5000, replace=False)
    if i == '13' or i =='4' or i =='6':
        sub = pd.DataFrame(binarize_sig(mat_total[(clusters == i) & (lengths_total >= 1000)])).rolling(5, axis=1, center=True, min_periods=1).mean().values
        idx = np.random.choice(sub.shape[0], 5000, replace=False)
        subsamp = sub[idx,:]
        for j in range(len(subsamp)):
            for k in range(len(subsamp[j])):
                print("%s\t%s\t%s\t%s\t%s" % (i ,j, k, subsamp[j][k], i), file=fho2)
fho2.close()


## Genome Composition

In [ ]:
#Create dataframe with TAD_labels, cluster_labels, replicate_labels
df = pd.DataFrame({'clusters':clusters,'labels':labels_total, 'reps':holes_total})
fho = open('fig4_stacked_bar_plot.txt','w')
for i in np.unique(clusters):
    cluster_frac_rep1 = len((df[(df['clusters'] == i) & (df['reps'] == 'Replicate1')])) / len(df[(df['reps'] == 'Replicate1')])
    cluster_frac_rep2 = len((df[(df['clusters'] == i) & (df['reps'] == 'Replicate2')])) / len(df[(df['reps'] == 'Replicate2')])
    print("%s\t%s\tReplicate 1" % (i,cluster_frac_rep1), file=fho)
    print("%s\t%s\tReplicate 2" % (i,cluster_frac_rep2), file=fho)
fho.close()

In [ ]:
print(dinucs_total[0])

## Dinucleotide Composition

In [ ]:
cluster_dinucs = []

for i in np.unique(clusters):
    if len(dinucs_total[clusters == i]) < 1000: continue
    dinucs_clust = np.sum(dinucs_total[clusters == i],axis=0) 
    cluster_dinucs.append(dinucs_clust)

cluster_dinucs = np.vstack(cluster_dinucs)
plt.figure(figsize=(10,10))
imshow(np.corrcoef(cluster_dinucs),interpolation=None, cmap=cm.RdBu)
colorbar()
plt.show()

## TAD Composition
We'd like to test the hypothesis that molecules from the same TADs significantly 
share (or don't share) molecular states. To test this, we're going to try two different
approaches. First, we'll make an information content calculation, asking whether each TAD has 
more or less information (in bits) compared to a random sampling of molecules. Second, we'll ask
whether the variance in computed NRLs for molecules taken from TADs is significantly different from the variance 
of computed NRLs from random molecules

In [ ]:

from collections import Counter

entropy_df = []
entropy_cntrl = [] 
#Entropy calc
for label in np.unique(labels_total):
    cluster_labs = clusters[labels_total == label]
    if len(cluster_labs) <= 50: continue
    control_labs = np.random.choice(clusters, size=len(cluster_labs))
#    print(len(cluster_labs))
    df_control = pd.DataFrame.from_dict(Counter(control_labs), orient='index')
    df = pd.DataFrame.from_dict(Counter(cluster_labs), orient='index')
#   print(df)
    df['norm'] = df[0] / df[0].sum()
    df_control['norm'] = df_control[0] / df_control[0].sum()
    entropy_df.append(scipy.stats.entropy(df['norm'].values))
    entropy_cntrl.append(scipy.stats.entropy(df_control['norm'].values))

    #print(entropy_df)
plt.figure(figsize=(10,10))
sns.distplot(entropy_df)
sns.distplot(entropy_cntrl)
plt.show()

In [ ]:
fho = open('Entropy_variance_figures.txt','w')
for i in range(len(entropy_df)):
    
    

In [ ]:
samp=[]
samp_frac = []
control=[]
control_frac = []
samp_lengths = []
#print(labels_total)
for label in np.unique(labels_total):
    samp.append(len(clusters[(labels_total == label) & (clusters != '16') & (clusters != '17')]))
    samp_lengths.append(np.average(lengths_total[(labels_total == label) & (clusters != '16') & (clusters != '17')]))

plt.figure(figsize=(10,10))
sns.scatterplot(x=samp)
plt.show()

    
# plt.figure(figsize=(10,10))
# sns.distplot(samp)
# plt.show()
#     num_unique = len(np.unique(cluster_labs))
#     cluster_filt = clusters[(clusters != '16') & (clusters != '17')]
# #   print(len(cluster_filt))
# #   print(len(cluster_labs))
#     control_labs = np.random.choice(cluster_filt, size=len(cluster_labs), replace=False)
#     num_unique_c = len(np.unique(control_labs))
#     samp.append(num_unique)
#     control.append(num_unique_c)

# print(len(samp))
# print(len(control))

# plt.figure(figsize=(10,10))
# sns.distplot(s)
# sns.distplot(control)
# plt.show()

In [ ]:
print(np.average(samp))

In [ ]:
fho=open('variance_plot2.txt', 'w')
for i in range(len(samp)):
    print("%s\t%s\tSample\tCluster Membership" % (i, samp[i]) , file = fho)
    print("%s\t%s\tControl\tCluster Membership" % (i, control[i]) , file = fho)
fho.close()

In [ ]:
def peak_variance(autocorrelogram):
    peaks = []
    non_peaks = 0
    for i in range(len(autocorrelogram)):
        mol = autocorrelogram[i]
        peak, data = signal.find_peaks(mol, height = 0.10, width = 25)
        if len(peak) == 0:
            non_peaks+=1
            continue
        peaks.append(peak[0])
    var = np.var(peaks)
    return (non_peaks, var)

samp=[]
samp_frac = []
control=[]
control_frac = []

for label in np.unique(labels_total):
    cluster_mols = auto_cor[labels_total == label]
    if len(cluster_mols) <= 50: continue
    control_mols = auto_cor[np.random.choice(auto_cor.shape[0], len(cluster_mols), replace=False),:]
    nonpeaks, var = peak_variance(cluster_mols)
    nonpeaks_c, var_c = peak_variance(control_mols)
    samp.append(var)
    control.append(var_c)
    samp_frac.append((nonpeaks / len(cluster_mols)))
    control_frac.append((nonpeaks_c / len(cluster_mols)))

    
plt.figure(figsize=(10,10))
sns.distplot(samp)
sns.distplot(control)
plt.show()

plt.figure(figsize=(10,10))
sns.distplot(samp_frac)
sns.distplot(control_frac)
plt.show()

#         plt.figure(figsize=(10,10))
#         subplot(121)
#         sns.distplot(peaks)
#         axvline(overall_peak)
#         subplot(122)
#         sns.distplot(length_sub,bins=50)
#         plt.show()

In [ ]:
fho = open('Entropy_variance_figures.txt','w')
for i in range(len(entropy_df)):
    print("%s\t%s\tSample\tEntropy" % (i, entropy_df[i]), file=fho)
for i in range(len(entropy_cntrl)):
    print("%s\t%s\tControl\tEntropy" % (i, entropy_cntrl[i]), file=fho)
for i in range(len(samp)):
    print("%s\t%s\tSample\tVariance" % (i, samp[i]), file=fho)
for i in range(len(control)):
    print("%s\t%s\tControl\tVariance" % (i, control[i]), file=fho)
fho.close()

In [ ]:
tad_zmw_coverage = []
for label in np.unique(labels_total):
    cluster_mols = len(auto_cor[labels_total == label])
    tad_zmw_coverage.append(cluster_mols)
print(len(tad_zmw_coverage))
plt.figure(figsize=(10,10))
sns.distplot(tad_zmw_coverage)
plt.show()

In [ ]:
test1 = scipy.stats.wilcoxon(entropy_df, entropy_cntrl,alternative='two-sided')
print(test1)
test2 = scipy.stats.wilcoxon(np.sqrt(samp), np.sqrt(control), alternative='two-sided')
print(test2)

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(tad_zmw_coverage)
plt.show()

## 3D Genome Analyses II: Domain Enrichment

In [ ]:
def distill_tipds_tad(tipds, valid_zmws, sitelist, length, label, subsample=0):
    '''Used to extract relevant mapped ZMWs given a list of ZMW ids. Also excludes bad ZMWs given the valid
    ZMW list. This is for fragment ends (doesn't search for a flat feaure of interest).'''
    sites = open(sitelist,'r')
    hole_nos = {}
    for i in sites:
        split = i.split()
        if int(split[0]) not in valid_zmws: continue
        hole_nos[split[0]] = [split[1], int(split[3]) - int(split[2]), split[4]]
    sites.close()
    new_mat = np.zeros(length)
    lengths = []
    labs = []
    reads = []
    n_mol = 0
    for hole in hole_nos:
        if subsample != 0:
            if n_mol > subsample: break
        read = tipds[int(hole)]
        dist = hole_nos[hole]
        if len(read) >= length:
            reads.append(read[:length])
            lengths.append(len(read))
            labs.append(dist[-1])
            n_mol +=1
    new_mat = np.vstack(reads)
    return (new_mat, lengths, labs)

length = 1000

tfs = open('./pbrun4_purple/biology_analyses/new_beds.txt')
#tfs = ['coopfoots\n']
lengths_total = np.array([0])
labels_total = np.array([0])
holes_total = np.array([0])
mat_total = []
mat_total_r = []
samp = 1000
#samp=250
for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    print(sample)
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    mat_rep1, mat_rep1_r, lengths, labels,holes, frac, tot = distill_tipds_bed(rep1, rep1_valid, sites_rep1, length, sample,subsample=samp)
    print(frac/tot)
    mat_rep2, mat_rep2_r, lengths2, labels2,holes2, frac, tot = distill_tipds_bed(rep2, rep2_valid, sites_rep2, length, sample,subsample=samp)    
    print(frac/tot)    
    mat_rep3, mat_rep3_r, lengths3, labels3,holes3, frac, tot = distill_tipds_bed(rep3, rep3_valid, sites_rep3, length, sample,subsample=samp*2)    
    print(frac/tot)    
    mat_rep4, mat_rep4_r, lengths4, labels4,holes4, frac, tot = distill_tipds_bed(rep4, rep4_valid, sites_rep4, length, sample,subsample=samp*2)    
    print(frac/tot)
    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4))
    mat_total_r.extend((mat_rep1_r,mat_rep2_r,mat_rep3_r,mat_rep4_r))
    print("%s\t%s" % (sample, (len(mat_rep1)+len(mat_rep2)+len(mat_rep3)+len(mat_rep4))))
#   print("%s_control\t%s" % (sample, (len(cntrl_mat_rep1)+len(cntrl_mat_rep2)+len(cntrl_mat_rep3)+len(cntrl_mat_rep4))))
    lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4))
    labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4))
    holes_total = np.concatenate((holes_total, holes, holes2, holes3, holes4))
#Also include Siva Satellite ZMWs
sites_rep1 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep2 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
sites_rep3 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep4 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
sample='Satellite'
mat_rep1, mat_rep1_r, lengths, labels, holes, frac, tot = distill_tipds_bed(rep1, rep1_valid, sites_rep1, length, sample,subsample=samp)
print(frac/tot)
mat_rep2, mat_rep2_r, lengths2, labels2, holes2, frac, tot = distill_tipds_bed(rep2, rep2_valid, sites_rep2, length, sample,subsample=samp)    
print(frac/tot)
mat_rep3, mat_rep3_r, lengths3, labels3, holes3, frac, tot = distill_tipds_bed(rep3, rep3_valid, sites_rep3, length, sample,subsample=samp*2)    
print(frac/tot)
mat_rep4, mat_rep4_r, lengths4, labels4, holes4, frac, tot = distill_tipds_bed(rep4, rep4_valid, sites_rep4, length, sample,subsample=samp*2)    
print(frac/tot)
mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4))
mat_total_r.extend((mat_rep1_r,mat_rep2_r,mat_rep3_r,mat_rep4_r))
print("%s\t%s" % (sample, (len(mat_rep1)+len(mat_rep2)+len(mat_rep3)+len(mat_rep4))))
lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4))
labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4))
holes_total = np.concatenate((holes_total, holes, holes2, holes3, holes4))

mat_total = np.vstack(mat_total)
mat_total_r = np.vstack(mat_total_r)
lengths_total = lengths_total[1:]
labels_total = labels_total[1:]
holes_total = holes_total[1:]
tfs.close()

In [ ]:
plt.figure(figsize=(10,10))
#plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'Satellite'], axis=0))
plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'H3K9me1'], axis=0))
plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'H3K27me3_random'], axis=0))
# plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'H3K36me3'], axis=0))
# plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'H3K9me3'], axis=0))
plt.show()

In [ ]:
print(mat_total)

In [ ]:
#Also include Siva Satellite ZMWs
sites_rep1 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep2 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt' % sample
sites_rep3 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep4 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
sample='Satellite'

In [ ]:
print(len(labels_total))
print(len(lengths_total))
print(len(mat_total))
print(len(auto_cor))
print(len(clusters))

## Clustering + Fisher's for NRLs Fig. 5

In [ ]:
def peak_cruncher2(ac1,ac2):
    peaks1 = []
    peaks2 = []
    for i in range(len(ac1)):
        mol1 = ac1[i]
        mol2 = ac2[i]
        peak1, data = signal.find_peaks(mol1, height = 0.10, width = 25)
        peak2, data = signal.find_peaks(mol2, height = 0.10, width = 25)
        if len(peak1) == 0:
            ac1p = np.nan
        else:
            ac1p = int(peak1[0])
        if len(peak2) == 0:
            ac2p = np.nan
        else:
            ac2p = int(peak2[0])
        peaks1.append(ac1p)
        peaks2.append(ac2p)
    peaks1 = np.array(peaks1)
    peaks2 = np.array(peaks2)
    med1 = np.nanmedian(peaks1)
    mad1 = scipy.stats.median_absolute_deviation(peaks1,nan_policy='omit')
    mis1 = np.count_nonzero(np.isnan(peaks1))
    med2 = np.nanmedian(peaks2)
    mad2 = scipy.stats.median_absolute_deviation(peaks2,nan_policy='omit')
    mis2 = np.count_nonzero(np.isnan(peaks2))
    return (peaks1, peaks2, med1, mad1,mis1, med2, mad2,mis2)

def smol_autos(auto_cor1, auto_cor2, clusters, fho):
    for i in np.unique(clusters):
        mat_new = auto_cor1[clusters == i]
        mat_new2 = auto_cor2[clusters == i]
        peaks1, peaks2, med1, mad1, mis1, med2, mad2, mis2 = peak_cruncher2(mat_new, mat_new2)
        mat_mean = np.mean(mat_new, axis=0)
        mat_mean2 = np.mean(mat_new2, axis=0)
        overall_peak, data = signal.find_peaks(mat_mean, height = 0.10, width = 25)
        overall_peak2, data = signal.find_peaks(mat_mean2, height = 0.10, width = 25)
        if len(overall_peak2) == 0:
            overall_peak2 = 0
        else:
            overall_peak2 = overall_peak2[0]
        if len(overall_peak) == 0:
            overall_peak = 0
        else:
            overall_peak = overall_peak[0]
        length_sub = lengths_total[clusters == i]
        if len(length_sub) <= 1000: continue
        print(len(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))]))
        print(len(peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))]))

        print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % \
              (i, overall_peak, overall_peak2, mis1, mis2, med1, med2, mad1, mad2, len(length_sub)))
        print("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % \
              (i, overall_peak, overall_peak2, mis1, mis2, med1, med2, mad1, mad2, len(length_sub)),file=fho
              )
        plt.figure(figsize=(5,5))
        sns.jointplot(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], kind="hex")
        plt.show()
        plt.figure(figsize=(5,5))
        sns.jointplot(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], length_sub[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], kind="hex")        
        plt.figure(figsize=(5,5))
        sns.jointplot(peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], length_sub[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], kind="hex")
#        plt.scatter(peaks1[(~np.isnan(peaks1)) & (~np.isnan(peaks2))], peaks2[(~np.isnan(peaks1)) & (~np.isnan(peaks2))],alpha=0.25)
        plt.show()

fho = open('test_out.txt','w')
nl = 1000
auto_cor, processed = process_xcors(np.nan_to_num(mat_total), max_len=nl, lengths=lengths_total)
auto_cor_r, processed = process_xcors(np.nan_to_num(mat_total_r), max_len=nl, lengths=lengths_total)
# # # processed = np.nan_to_num(pd.DataFrame(mat_total).rolling(5,axis=1, center=True, min_periods=1).mean())
print(len(auto_cor))
clusters, sp_obj = cluster_mats(auto_cor[:,:500],res=0.40,neighbors=15)
plotter(binarize_sig(mat_total_r), auto_cor, clusters, 5)
plotter(binarize_sig(mat_total), auto_cor, clusters, 5)
smol_autos(auto_cor, auto_cor_r, clusters, fho)
fho.close()

In [ ]:
print(len(auto_cor[0]))

## Plot specific averages for certain clusters, then plot specific cluster of single molecules for effect

In [ ]:
avg1 = pd.Series(np.nanmean(binarize_sig(mat_total[(labels_total == 'H3K36me3') & (clusters == '2')]),axis=0)).rolling(20,center=True, min_periods=1).mean()[::-1]
avg2 = pd.Series(np.nanmean(binarize_sig(mat_total[(labels_total == 'H3K9me3') & (clusters == '2')]),axis=0)).rolling(20,center=True, min_periods=1).mean()[::-1]
avg3 =pd.Series(np.nanmean(binarize_sig(mat_total[(labels_total == 'H3K4me3') & (clusters == '5')]),axis=0)).rolling(20,center=True, min_periods=1).mean()[::-1]
avg4 =pd.Series(np.nanmean(binarize_sig(mat_total[(labels_total == 'H3K27me3') & (clusters == '7')]),axis=0)).rolling(20,center=True, min_periods=1).mean()[::-1]
avg5 =pd.Series(np.nanmean(binarize_sig(mat_total[(labels_total == 'Satellite') & (clusters == '0')]),axis=0)).rolling(20,center=True, min_periods=1).mean()[::-1]

fho = open('fig5_lineplots_v2.txt','w')
for i in range(len(avg1)):
    label = 'H3K36me3'
    print("%s\t%s\t%s" % (i, avg1[i], label),file=fho)
    label = 'H3K9me3'
    print("%s\t%s\t%s" % (i, avg2[i], label),file=fho)
    label = 'H3K4me3'
    print("%s\t%s\t%s" % (i, avg3[i], label),file=fho)
    label = 'H3K27me3'
    print("%s\t%s\t%s" % (i, avg4[i], label),file=fho)
    label = 'Satellite'
    print("%s\t%s\t%s" % (i, avg5[i], label),file=fho)
fho.close()
plt.figure(figsize=(10,10))
plt.plot(range(len(avg1)),avg1)
plt.plot(range(len(avg2)),avg2)
plt.plot(range(len(avg3)),avg3)
plt.plot(range(len(avg4)),avg4)

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
#plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'Satellite'], axis=0)5
plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[(labels_total == 'H3K36me3') & (clusters == '2')], axis=0))
plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[(labels_total == 'H3K9me3') & (clusters == '2')], axis=0))
plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[(labels_total == 'H3K4me3') & (clusters == '5')], axis=0))
plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[(labels_total == 'H3K27me3') & (clusters == '7')], axis=0))

# plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'H3K36me3'], axis=0))
# plt.plot(range(len(np.nanmean(mat_total, axis=0))), np.nanmean(mat_total[labels_total == 'H3K9me3'], axis=0))
plt.show()

## Heatmap representation of clusters for Fig 5


In [ ]:
fho1 = open('hmap_figure_fig5.txt','w')
fho2 = open('hmap_figure_fig5_smoothed.txt','w')
for i in np.unique(clusters):
    mat_new = auto_cor[clusters == i]
    if len(mat_new) < 100: continue
    plotter = np.nanmean(mat_new, axis=0)
    mat_new2 = mat_total[clusters == i]
    hmap = pd.Series(np.nanmean(binarize_sig(mat_new2),axis=0)).rolling(20,center=True, min_periods=1).mean()
    for j in range(len(plotter)):
        print("%s\t%s\t%s" % (i, j, plotter[j]), file=fho1)
    for j in range(len(hmap)):
        print("%s\t%s\t%s" % (i, j, hmap[j]), file=fho2)        
    process = pd.DataFrame(binarize_sig(mat_new2)).rolling(5, axis=1, center=True, min_periods=1).mean().values
    plt.figure(figsize=(5,5))
    plt.plot(range(len(mat_new2[0])), np.nanmean(process, axis=0))
    xlabel('Distance to 5\'-MNase cut')
    ylabel('Average smoothed IPD (frames)')
    subplot(143)
    imshow(mat_new, interpolation=None, cmap=cm.BuPu)
    xlabel('Offset (bp)')
    subplot(144)
    imshow(process, interpolation=None, cmap=cm.BuPu)
    xlabel('Distance')
fho1.close()
fho2.close()

## NRL Estimates for Figure 5

In [ ]:
def peak_cruncher(autocorrelogram):
    peaks = []
    heights = []
    for i in range(len(autocorrelogram)):
        mol = autocorrelogram[i]
        peak, data = signal.find_peaks(mol, height = 0.10, width = 25)
        if len(peak) == 0: continue
        peaks.append(peak[0])
        heights.append(data['peak_heights'][0])
    #for after I implement filtration via controls
#         else:
#             peaks.append(0)
#             heights.append(0)
    med = np.median(peaks)
    mod = scipy.stats.mode(peaks)
    return (peaks, heights, med, mod)

def plotter_v2(new_mat, auto_cor, clusters):
    for i in np.unique(clusters):
        print(i)
        mat_new = auto_cor[clusters == i]
        peaks, heights, med, mod = peak_cruncher(mat_new)
        mat_mean = np.mean(mat_new, axis=0)
        overall_peak, data = signal.find_peaks(mat_mean, height = 0.08, width = 20)
        if len(overall_peak) == 0:
            overall_peak = 0
        else:
            overall_peak = overall_peak
        print(overall_peak)
        length_sub = lengths_total[clusters == i]
        plt.figure(figsize=(10,10))
        subplot(121)
        sns.distplot(peaks)
        axvline(overall_peak)
        subplot(122)
        sns.distplot(length_sub,bins=50)
        plt.show()
plotter_v2(mat_total, auto_cor, clusters)

In [ ]:
## Find Cluster N molecules and plot the long ones. 
#create dataframe of clusters, hole_nos, lengths
df = pd.DataFrame({'clusters':clusters, 'holes':holes_total, 'lengths':lengths_total,'labels':labels_total})
#Cluster 6 holes
df_6 = df[(df['clusters'] == '2') & (df['lengths'] >= 1000) & (df['labels'] == 'H3K9me3')]['holes'].values
print(df_6)

In [ ]:
auto_cor, processed = process_xcors(np.nan_to_num(mat_total), length)
processed = np.nan_to_num(pd.DataFrame(mat_total).rolling(5,axis=1, center=True, min_periods=1).mean())
print(len(auto_cor))
motif_sites = processed[:,int(length / 2) - 10:int(length / 2) + 10]
clusters, sp_obj = cluster_mats(auto_cor,res=0.5,neighbors=15)
plotter(binarize_sig(mat_total), auto_cor, clusters, 33)

In [ ]:
import scipy as sp
#just do the damn fishers tests in pandas
#fho = open('fishers_tests_fig5.txt','w')
fho = open('fishers_tests_figs5_sampled.txt','w')
cluster_df = {'clusters':clusters, 'labels':labels_total}
cluster_df = pd.DataFrame(cluster_df)
for i in np.unique(clusters):
    if len(labels_total[clusters == i]) < 100:
        plot = 0
    else:
        plot = 1
    for j in np.unique(labels_total):
        num_clust_lab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] == j)])
        num_clust_notlab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] != j)])
        num_notclust_lab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] == j)])
        num_notclust_notlab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval,plot), file=fho)
fho.close()
#labs = np.array(labels)
#print("Cluster\t%s\t%s\tOR" % (sample2, sample1))

In [ ]:
fho1 = open('hmap_figure_sfig5.txt','w')
fho2 = open('hmap_figure_sfig5_smoothed.txt','w')
for i in np.unique(clusters):
    mat_new = auto_cor[clusters == i]
    if len(mat_new) < 100: continue
    plotted = np.nanmean(mat_new, axis=0)
    mat_new2 = mat_total[clusters == i]
    hmap = pd.Series(np.nanmean(binarize_sig(mat_new2),axis=0)).rolling(20,center=True, min_periods=1).mean()
    for j in range(len(plotted)):
        print("%s\t%s\t%s" % (i, j, plotted[j]), file=fho1)
    for j in range(len(hmap)):
        print("%s\t%s\t%s" % (i, j, hmap[j]), file=fho2)        
    process = pd.DataFrame(binarize_sig(mat_new2)).rolling(5, axis=1, center=True, min_periods=1).mean().values
    plt.figure(figsize=(5,5))
    plt.plot(range(len(mat_new2[0])), np.nanmean(process, axis=0))
    xlabel('Distance to 5\'-MNase cut')
    ylabel('Average smoothed IPD (frames)')
    subplot(143)
    imshow(mat_new, interpolation=None, cmap=cm.BuPu)
    xlabel('Offset (bp)')
    subplot(144)
    imshow(process, interpolation=None, cmap=cm.BuPu)
    xlabel('Distance')
fho1.close()
fho2.close()

In [ ]:
def peak_cruncher(autocorrelogram):
    peaks = []
    heights = []
    for i in range(len(autocorrelogram)):
        mol = autocorrelogram[i]
        peak, data = signal.find_peaks(mol, height = 0.10, width = 25)
        if len(peak) == 0: continue
        peaks.append(peak[0])
        heights.append(data['peak_heights'][0])
    #for after I implement filtration via controls
#         else:
#             peaks.append(0)
#             heights.append(0)
    med = np.median(peaks)
    mod = scipy.stats.mode(peaks)
    return (peaks, heights, med, mod)

def plotter_v2(new_mat, auto_cor, clusters):
    for i in np.unique(clusters):
        print(i)
        mat_new = auto_cor[clusters == i]
        peaks, heights, med, mod = peak_cruncher(mat_new)
        mat_mean = np.mean(mat_new, axis=0)
        overall_peak, data = signal.find_peaks(mat_mean, height = 0.10, width = 25)
        if len(overall_peak) == 0:
            overall_peak = 0
        else:
            overall_peak = overall_peak[0]
        print(overall_peak)
        length_sub = lengths_total[clusters == i]
        plt.figure(figsize=(10,10))
        subplot(121)
        sns.distplot(peaks)
        axvline(overall_peak)
        subplot(122)
        sns.distplot(length_sub,bins=50)
        plt.show()
plotter_v2(mat_total, auto_cor, clusters)

In [ ]:
print(mat_total)

In [ ]:
import scipy as sp
#just do the damn fishers tests in pandas
#fho = open('fishers_tests_fig5.txt','w')
fho = open('fishers_tests_figs5_sampled.txt','w')
cluster_df = {'clusters':clusters, 'labels':labels_total}
cluster_df = pd.DataFrame(cluster_df)
for i in np.unique(clusters):
    if len(labels_total[clusters == i]) < 100:
        plot = 0
    else:
        plot = 1
    for j in np.unique(labels_total):
        num_clust_lab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] == j)])
        num_clust_notlab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] != j)])
        num_notclust_lab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] == j)])
        num_notclust_notlab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval,plot), file=fho)
fho.close()
#labs = np.array(labels)
#print("Cluster\t%s\t%s\tOR" % (sample2, sample1))

In [ ]:
def peak_cruncher(autocorrelogram):
    peaks = []
    heights = []
    for i in range(len(autocorrelogram)):
        mol = autocorrelogram[i]
        peak, data = signal.find_peaks(mol, height = 0.10, width = 30)
        if len(peak) == 0: continue
        peaks.append(peak[0])
        heights.append(data['peak_heights'][0])
    #for after I implement filtration via controls
#         else:
#             peaks.append(0)
#             heights.append(0)
    med = np.median(peaks)
    mod = scipy.stats.mode(peaks)
    return (peaks, heights, med, mod)

def plotter_v2(new_mat, auto_cor, clusters):
    for i in np.unique(clusters):
        print(i)
        mat_new = auto_cor[clusters == i]
        peaks, heights, med, mod = peak_cruncher(mat_new)
        mat_mean = np.mean(mat_new, axis=0)
        overall_peak, data = signal.find_peaks(mat_mean, height = 0.15, width = 20)
        if len(overall_peak) == 0:
            overall_peak = 0
        else:
            overall_peak = overall_peak
        print(overall_peak)
        length_sub = lengths_total[clusters == i]
        plt.figure(figsize=(10,10))
        subplot(121)
        sns.distplot(peaks)
        axvline(overall_peak)
        subplot(122)
        sns.distplot(length_sub,bins=50)
        plt.show()
plotter_v2(mat_total, auto_cor, clusters)

## GC content and dinucleotide bias for clusters

In [ ]:
nuc_comp = './pbrun4_purple/'
for line in nuc_comp:
    

In [ ]:
for i in np.unique(clusters):
    length_sub = lengths_total[clusters == i]
    plt.figure(figsize=(5,5))
    sns.distplot(length_sub,bins=100)
    plt.show()

In [ ]:
from collections import Counter
foo = ['apple','apple','orange'] 
print(Counter(foo))
df = pd.DataFrame.from_dict(Counter(foo), orient='index')
print(df)
df['norm'] = df[0] / df[0].sum()
print(scipy.stats.entropy(df['norm'].values))
#for label in np.unique(labels_total):
    

In [ ]:
test = [0,0,0,0,0,0,0,1]


In [ ]:
from collections import Counter

entropy_df = []
entropy_cntrl = [] 
#Entropy calc
for label in np.unique(labels_total):
    cluster_labs = clusters[labels_total == label]
    if len(cluster_labs) <= 50: continue
    control_labs = np.random.choice(clusters, size=len(cluster_labs))
#    print(len(cluster_labs))
    df_control = pd.DataFrame.from_dict(Counter(control_labs), orient='index')
    df = pd.DataFrame.from_dict(Counter(cluster_labs), orient='index')
#   print(df)
    df['norm'] = df[0] / df[0].sum()
    df_control['norm'] = df_control[0] / df_control[0].sum()
    entropy_df.append(scipy.stats.entropy(df['norm'].values))
    entropy_cntrl.append(scipy.stats.entropy(df_control['norm'].values))

In [ ]:
#print(entropy_df)
plt.figure(figsize=(10,10))
sns.distplot(entropy_df)
sns.distplot(entropy_cntrl)
plt.show()

In [ ]:
# #Load up scanpy objects
# labels = np.array(labels_total)
# #need to recast NaNs as something, we'll say 0 here, which seems defensible.
# mat_ann = scanpy.AnnData(X=auto_cor) 
# mat_ann.obs.index = labels
# #sample labels for plotting
# mat_ann.obs['samples'] = labels
# scanpy.tl.pca(mat_ann)
# scanpy.pp.neighbors(mat_ann)
# #we'll use leiden clustering (state-of-the-art community detection)
scanpy.tl.leiden(mat_ann)
# scanpy.tl.umap(mat_ann)
#mat_ann.obs['colours'] = color_labels
fig, ax = plt.subplots(figsize=(20, 14))
scanpy.pl.umap(mat_ann, color='leiden', ax=ax)

## Use strand information + fragment end info to make metaplots from MNase data

In [ ]:
sample1 = 'CAGE_bed_flat'
test_file1 = open('./pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample1, 'r')
test_file2 = open('./pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample1, 'r')
test_file3 = open('./pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample1, 'r')
test_file4 = open('./pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample1, 'r')

files = [test_file1,test_file2,test_file3,test_file4]
#files = [test_file1,test_file2]
length = 2000
meta = np.zeros(length)
for test_file in files:
    for line in test_file:
        split = line.split()
        start = int(split[2])
        end = int(split[3])
        frag = int(split[4])
        if split[-1] == '+':
            dist1 = start - frag + int(length / 2)
            dist2 = end - frag + int(length / 2)
            if 0 <= dist1 < length:
                meta[dist1] += 1
            if 0 <= dist2 < length:
                meta[dist2] += 1
        else:
            dist1 = frag - start + int(length / 2)
            dist2 = frag - end + int(length / 2)
            if 0 <= dist1 < length:
                meta[dist1] += 1
            if 0 <= dist2 < length:
                meta[dist2] += 1

meta=pd.Series(meta).rolling(33,center=True, min_periods=1).mean()
plt.figure(figsize=(10,10))
plt.plot(range(len(meta)), meta)
plt.show()

for test_file in files:
    test_file.close()

## Figure 3 Transcription Factors I: MNase Cuts

In [ ]:
tfs = open('./pbrun4_purple/biology_analyses/tfs.txt')
#tfs = ['CTCF\n']
tot_zmws = 247302 + 226280 + 732496 + 747401
for line in tfs:
    tf = "%s_flat" % (line.split()[0])
    test_file1 = open('./pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    test_file2 = open('./pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    test_file3 = open('./pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    test_file4 = open('./pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % tf, 'r')
    files = [test_file1,test_file2,test_file3,test_file4]
    #files = [test_file1,test_file2]
    length = 1000
    meta = np.zeros(length)
    for test_file in files:
        for line in test_file:
            split = line.split()
            start = int(split[2])
            end = int(split[3])
            frag = int(split[4])
            if split[-1] == '+':
                dist1 = start - frag + int(length / 2)
                dist2 = end - frag + int(length / 2)
                if 0 <= dist1 < length:
                    meta[dist1] += 1
                if 0 <= dist2 < length:
                    meta[dist2] += 1
            else:
                dist1 = frag - start + int(length / 2)
                dist2 = frag - end + int(length / 2)
                if 0 <= dist1 < length:
                    meta[dist1] += 1
                if 0 <= dist2 < length:
                    meta[dist2] += 1
    meta=pd.Series(meta).rolling(5,center=True, min_periods=1).mean()
    plt.figure(figsize=(10,10))
    plt.plot(range(len(meta)), meta)
    plt.show()
    for test_file in files:
        test_file.close()
    fho = open('%s_MNase.txt' % tf,'w')
    for i in range(len(meta)):
        print("%s\t%s\t%s" % (i, meta[i] / tot_zmws * 1000000, tf), file=fho)
    fho.close()

## Figure 3 Transcription Factors II: m6dA signal

In [ ]:
length = 500

tfs = open('./pbrun4_purple/biology_analyses/tfs.txt')
#tfs = ['CTCF\n']
lengths_total = np.array([0])
labels_total = np.array([0])
mat_total = []

for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    cntrl_sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    mat_rep1, lengths, labels = distill_tipds_flat_centered(rep1, rep1_valid, sites_rep1, length, sample)
    mat_rep2, lengths2, labels2 = distill_tipds_flat_centered(rep2, rep2_valid, sites_rep2, length, sample)
    mat_rep3, lengths3, labels3 = distill_tipds_flat_centered(rep3, rep3_valid, sites_rep3, length, sample)
    mat_rep4, lengths4, labels4 = distill_tipds_flat_centered(rep4, rep4_valid, sites_rep4, length, sample)
    cntrl_mat_rep1, cntrl_lengths, cntrl_labels = distill_tipds_flat_centered(rep1, rep1_valid, cntrl_sites_rep1, length, "%s_control" % sample ,subsample=len(lengths))        
    cntrl_mat_rep2, cntrl_lengths2, cntrl_labels2 = distill_tipds_flat_centered(rep2, rep2_valid, cntrl_sites_rep2, length, "%s_control" % sample,subsample=len(lengths2))    
    cntrl_mat_rep3, cntrl_lengths3, cntrl_labels3 = distill_tipds_flat_centered(rep3, rep3_valid, cntrl_sites_rep3, length, "%s_control" % sample,subsample=len(lengths3))        
    cntrl_mat_rep4, cntrl_lengths4, cntrl_labels4 = distill_tipds_flat_centered(rep4, rep4_valid, cntrl_sites_rep4, length, "%s_control" % sample,subsample=len(lengths4))    
    mat_msix = np.vstack((mat_rep1, mat_rep2, mat_rep3, mat_rep4))
    mat_msix_control = np.vstack((cntrl_mat_rep1, cntrl_mat_rep2, cntrl_mat_rep3, cntrl_mat_rep4))
    process = pd.Series(np.nanmean(binarize_sig(mat_msix), axis=0)).rolling(20,center=True,min_periods=1).mean()
    process_neg = pd.Series(np.nanmean(binarize_sig(mat_msix_control), axis=0)).rolling(20,center=True,min_periods=1).mean()
#     process = np.nanmean(binarize_sig(pd.DataFrame(mat_msix).rolling(10, axis=1, center=True, min_periods=1).mean()), axis=0)
#     process_neg = np.nanmean(binarize_sig(pd.DataFrame(mat_msix_control).rolling(10, axis=1, center=True, min_periods=1).mean()), axis=0)
    plt.figure(figsize=(10,10))
    subplot(121)
    plt.plot(range(len(process)), process)
    subplot(122)
    plt.plot(range(len(process_neg)), process_neg)
    plt.show()
    fho=open('%s_m6A.txt' % tf,'w')
    for i in range(len(process)):
        print("%s\t%s\t%s\tsite" % (i, process[i], tf),file=fho)
        print("%s\t%s\t%s\tcontrol" % (i, process_neg[i], tf),file=fho)
    fho.close()
    
    

tfs.close()

In [ ]:
ctcf_m6da = np.nanmean(process1, axis=0)
myc_m6da = np.nanmean(process2, axis=0)
fho=open('ctcf_m6a.txt','w')
for i in range(len(ctcf_m6da)):
    print("%s\t%s\tCTCF" % (i, ctcf_m6da[i]),file=fho)
fho.close()

fho=open('myc_m6a.txt','w')
for i in range(len(ctcf_m6da)):
    print("%s\t%s\tc-MYC" % (i, ctcf_m6da[i]),file=fho)
fho.close()

## Working on a version of distill_flat that centers the molecules.

In [ ]:
def distill_tipds_flat_centered(tipds, valid_zmws, sitelist, length, label):
    sites = open(sitelist,'r')
    hole_nos = {}
    for i in sites:
        split = i.split()
        if int(split[0]) not in valid_zmws: continue
        #check to make sure the feature is in the middle of the read 
        #and enough on both sites
        if int(split[2]) <= int(split[4]) <= int(split[3]):
            index1 = int(split[4]) - int(split[2])
            index2 = int(split[3]) - int(split[4])
            strand = split[-1]
            if index1 > (length / 2) and index2 > (length / 2):
                hole_nos[split[0]] = (index1, strand)
    sites.close()
    new_mat = np.zeros(length)
    lengths = []
    labs = []
    for hole in hole_nos:
        read = tipds[int(hole)]
        index,strand = hole_nos[hole]
        if strand == "+":
            extract = tipds[int(hole)][int(index - (length / 2)): int(index + (length / 2))]
            if len(extract) != length: continue
            new_mat = np.vstack((new_mat, extract))
            labs.append(label)
            lengths.append(len(tipds[int(hole)]))
        else:
            extract = tipds[int(hole)][::-1][int(index - (length / 2)):int(index + (length / 2))]
            if len(extract) != length: continue
            new_mat = np.vstack((new_mat, extract))
            labs.append(label)
            lengths.append(len(tipds[int(hole)]))
    return (new_mat[1:], np.array(lengths), np.array(labs))

## Testing out the new centered function

In [ ]:
length = 500
#BINARIZED
k562_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_chromatin_bingmm'
k562_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_chromatin_rep2_bingmm'

k562_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_chromatin_bingmm'
k562_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_chromatin_rep2_bingmm'

# k562_neg_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_neg_bingmm'
# k562_neg_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/binarized/pbrun4_purple_k562_postlyso_neg_rep2_bingmm'

# k562_neg_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_neg_bingmm'
# k562_neg_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/binarized/pbrun6_k562_postlyso_neg_rep2_bingmm'

rep1, rep1_valid = eat_pickle_binary("%s.pickle" % k562_rep1)
rep2, rep2_valid = eat_pickle_binary("%s.pickle" % k562_rep2)
rep3, rep3_valid = eat_pickle_binary("%s.pickle" % k562_rep3)
rep4, rep4_valid = eat_pickle_binary("%s.pickle" % k562_rep4)



#NONBINARIZED
# k562_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_chromatin_onlyT'
# k562_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_chromatin_rep2_onlyT'

# k562_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_chromatin_onlyT'
# k562_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_chromatin_rep2_onlyT'

# # k562_neg_rep1 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_neg_onlyT'
# # k562_neg_rep2 = '/avicenna/vramani/analyses/pacbio/pbrun4_purple/processed/onlyT/pbrun4_purple_k562_postlyso_neg_rep2_onlyT'

# # k562_neg_rep3 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_neg_onlyT'
# # k562_neg_rep4 = '/avicenna/vramani/analyses/pacbio/pbrun6/processed/onlyT/pbrun6_k562_postlyso_neg_rep2_onlyT'

# rep1, rep1_valid = eat_pickle("%s.pickle" % k562_rep1, "%s_zmwinfo.pickle" % k562_rep1)
# rep2, rep2_valid = eat_pickle("%s.pickle" % k562_rep2, "%s_zmwinfo.pickle" % k562_rep2)
# rep3, rep3_valid = eat_pickle("%s.pickle" % k562_rep3, "%s_zmwinfo.pickle" % k562_rep3)
# rep4, rep4_valid = eat_pickle("%s.pickle" % k562_rep4, "%s_zmwinfo.pickle" % k562_rep4)


sample1 = 'TSS_human_exprs.flat.unexpressed_filtered_flat'
sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample1
sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample1
sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample1
sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample1

# sites_neg_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_neg--k562_postlyso_neg.ccs.aligned.sorted.bam.%s_zmws' % sample1
# sites_neg_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_neg_rep2--k562_postlyso_neg_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample1

sample2 = 'TSS_human_exprs.flat.0.8_filtered_flat' 
cntrl_sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample2
cntrl_sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample2
cntrl_sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample2
cntrl_sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample2

# cntrl_sites_neg_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_neg--k562_postlyso_neg.ccs.aligned.sorted.bam.%s_zmws' % sample2
# cntrl_sites_neg_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_neg_rep2--k562_postlyso_neg_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample2



# neg_rep1, neg_rep1_valid = eat_pickle("%s.pickle" % k562_neg_rep1, "%s_zmwinfo.pickle" % k562_neg_rep1)
# neg_rep2, neg_rep2_valid = eat_pickle("%s.pickle" % k562_neg_rep2, "%s_zmwinfo.pickle" % k562_neg_rep2)

mat_rep1, lengths, labels = distill_tipds_flat_centered(rep1, rep1_valid, sites_rep1, length, sample1)
mat_rep2, lengths2, labels2 = distill_tipds_flat_centered(rep2, rep2_valid, sites_rep2, length, sample1)
mat_rep3, lengths3, labels3 = distill_tipds_flat_centered(rep3, rep3_valid, sites_rep3, length, sample1)
mat_rep4, lengths4, labels4 = distill_tipds_flat_centered(rep4, rep4_valid, sites_rep4, length, sample1)

mat_rep5, lengths5, labels5 = distill_tipds_flat_centered(rep1, rep1_valid, cntrl_sites_rep1, length, sample2)
mat_rep6, lengths6, labels6 = distill_tipds_flat_centered(rep2, rep2_valid, cntrl_sites_rep2, length, sample2)
mat_rep7, lengths7, labels7 = distill_tipds_flat_centered(rep3, rep3_valid, cntrl_sites_rep3, length, sample2)
mat_rep8, lengths8, labels8 = distill_tipds_flat_centered(rep4, rep4_valid, cntrl_sites_rep4, length, sample2)


# neg_mat_rep1, l1, labels5 = distill_tipds_flat(neg_rep1, neg_rep1_valid, sites_neg_rep1, length, "%s_neg" % sample1)
# neg_mat_rep2, l1, labels6 = distill_tipds_flat(neg_rep2, neg_rep2_valid, sites_neg_rep2, length, "%s_neg" % sample1)
# neg_mat_rep3, l1, labels7 = distill_tipds_flat(neg_rep1, neg_rep1_valid, cntrl_sites_neg_rep1, length, "%s_neg" % sample2)
# neg_mat_rep4, l1, labels8 = distill_tipds_flat(neg_rep2, neg_rep2_valid, cntrl_sites_neg_rep2, length, "%s_neg" % sample2)


#mat_total = np.vstack((mat_rep1,mat_rep2,mat_rep3,mat_rep4, neg_mat_rep1, neg_mat_rep2, neg_mat_rep3, neg_mat_rep4))
mat_total = np.vstack((mat_rep1,mat_rep2,mat_rep3,mat_rep4,mat_rep5,mat_rep6,mat_rep7,mat_rep8))
mat_samp1 = np.vstack((mat_rep1,mat_rep2, mat_rep3,mat_rep4))
mat_samp2 = np.vstack((mat_rep5,mat_rep6, mat_rep7,mat_rep8))


# mat_samp1 = np.vstack((mat_rep1,mat_rep2,mat_rep3,mat_rep4))
# mat_samp2 = np.vstack((mat_rep5,mat_rep6,mat_rep7,mat_rep8))                      
mat_rep1,mat_rep2,mat_rep3,mat_rep4,mat_rep5,mat_rep6,mat_rep7,mat_rep8 = [0,0,0,0,0,0,0,0]

lengths = np.concatenate((lengths, lengths2,lengths3,lengths4,lengths5,lengths6,lengths7,lengths8))
#labels = np.concatenate((labels, labels2,labels5,labels6))
labels = np.concatenate((labels, labels2,labels3,labels4,labels5,labels6,labels7,labels8))
#lengths2, labels2, lengths3, labels3, lengths4, labels4 = (0,0,0,0,0,0)


In [ ]:
process1 = pd.DataFrame(mat_samp1).rolling(33,axis=1, center=True, min_periods=1).mean()
process2 = pd.DataFrame(mat_samp2).rolling(33,axis=1, center=True, min_periods=1).mean()


plt.figure(figsize=(10,5))
subplot(121)
plt.plot(range(len(mat_samp1[0])), np.nanmean(process1, axis=0))
subplot(122)
plt.plot(range(len(mat_samp2[0])), np.nanmean(process2, axis=0))
plt.show()

In [ ]:
auto_cor = process_xcors(mat_total, 500)
clusters = cluster_mats(mat_total)
plotter(mat_total, auto_cor, clusters)

labs = np.array(labels)
print("Cluster\t%s\t%s\tOR" % (sample2, sample1))
for i in np.unique(clusters):
    sub = labs[clusters == i]
    num_control = len(sub[sub == sample2]) / len(labs[labs == sample2])
    tot1 += num_control
    num_ctcf = len(sub[sub == sample1]) / len(labs[labs==sample1])
    tot2 += num_ctcf
    #num_neg = len(sub[sub == '%s_neg' % sample1]) / len(labs[labs=='%s_neg' % sample1])
    lo = np.log(num_ctcf / num_control)
    #num_neg2 = len(sub[sub == '%s_neg' % sample2]) / len(labs[labs=='%s_neg' % sample2])
    #print("%s\t%s\t%s\t%s\t%s\t%s" % (i, num_control, num_ctcf, num_neg, num_neg2, lo))
    print("%s\t%s\t%s\t%s" % (i, num_control, num_ctcf,lo)) 

In [ ]:
#We'd like to sample molecules so that we have ~1000 per sequencing run per replicate per domain
length = 500

tfs = open('./pbrun4_purple/biology_analyses/all_bed_files.txt')
lengths_total = np.array([0])
labels_total = np.array([0])
mat_total = np.zeros(length)
for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    mat_rep1, lengths, labels = distill_tipds_bed(rep1, rep1_valid, sites_rep1, length, sample, subsample=1000)
    mat_rep2, lengths2, labels2 = distill_tipds_bed(rep2, rep2_valid, sites_rep2, length, sample,subsample=1000)
    mat_rep3, lengths3, labels3 = distill_tipds_bed(rep3, rep3_valid, sites_rep3, length, sample,subsample=1000)
    mat_rep4, lengths4, labels4 = distill_tipds_bed(rep4, rep4_valid, sites_rep4, length, sample,subsample=1000)
    mat_total = np.vstack((mat_total, mat_rep1,mat_rep2,mat_rep3,mat_rep4))
    lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4))
    labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4))

mat_total = mat_total[1:]
lengths_total = lengths_total[1:]
labels_total = labels_total[1:]

tfs.close()



## Troubleshooting TF code

In [ ]:
def distill_tipds_flat_centered_test(tipds, valid_zmws, sitelist, length, label,subsample=0):
    sites = open(sitelist,'r')
    hole_nos = {}
    for i in sites:
        split = i.split()
        if int(split[0]) not in valid_zmws: continue
        #check to make sure the feature is in the middle of the read 
        #and enough on both sites
        if int(split[2]) <= int(split[4]) <= int(split[3]):
            index1 = int(split[4]) - int(split[2])
            index2 = int(split[3]) - int(split[4])
            strand = split[-1]
            if index1 > (length / 2) and index2 > (length / 2):
                hole_nos[split[0]] = (index1, strand, int(split[2]), int(split[3]))
    sites.close()
    lengths = []
    labs = []
    reads = []
    mno = 0
    for hole in hole_nos:
        read = tipds[int(hole)]
        index,strand, start, end = hole_nos[hole]
        print("%s\t%s" % (len(read), (end-start)))
        if strand == "+":
            extract = tipds[int(hole)][int(index - (length / 2)): int(index + (length / 2))]
            if len(extract) != length: continue
            reads.append(extract)
            labs.append(label)
            lengths.append(len(tipds[int(hole)]))
        else:
            extract = tipds[int(hole)][::-1][int(index - (length / 2)):int(index + (length / 2))]
            if len(extract) != length: continue
            reads.append(extract)
            labs.append(label)
            lengths.append(len(tipds[int(hole)]))
        if subsample != 0:
            mno += 1
            if mno == subsample: break
    new_mat = np.vstack(reads)
    return (new_mat, np.array(lengths), np.array(labs))

length = 500

#tfs = open('./pbrun4_purple/biology_analyses/tfs.txt')
tfs = ['REST\n']
lengths_total = np.array([0])
labels_total = np.array([0])
mat_total = []

for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    mat_rep1, lengths, labels = distill_tipds_flat_centered_test(rep1, rep1_valid, sites_rep1, length, sample)    
    mat_rep2, lengths2, labels2 = distill_tipds_flat_centered_test(rep2, rep2_valid, sites_rep2, length, sample)
    mat_rep3, lengths3, labels3 = distill_tipds_flat_centered_test(rep3, rep3_valid, sites_rep3, length, sample)
    mat_rep4, lengths4, labels4 = distill_tipds_flat_centered_test (rep4, rep4_valid, sites_rep4, length, sample)
    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4,cntrl_mat_rep1,cntrl_mat_rep2,cntrl_mat_rep3,cntrl_mat_rep4))
#    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4))

    print("%s\t%s" % (sample, (len(mat_rep1)+len(mat_rep2)+len(mat_rep3)+len(mat_rep4))))
    print("%s_control\t%s" % (sample, (len(cntrl_mat_rep1)+len(cntrl_mat_rep2)+len(cntrl_mat_rep3)+len(cntrl_mat_rep4))))
    lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4, cntrl_lengths, cntrl_lengths2,cntrl_lengths3,cntrl_lengths4))
    labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4,cntrl_labels, cntrl_labels2, cntrl_labels3, cntrl_labels4))
#     lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4))
#     labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4))
    
mat_total = np.vstack(mat_total)
lengths_total = lengths_total[1:]
labels_total = labels_total[1:]

tfs.close()

# Analyzing Stergachis et al data for concordance

In [ ]:
import pickle

def eat_pickle_filt(pick, samp_label,length, valid_zmws, min_len = 500, samp= 0):
    valid = {}
    zmws = open(valid_zmws)
    for line in zmws:
        split = line.split()
        valid[split[0]] = split[-1]
    #print(valid)
    with open(pick, 'rb') as fout:
        tipds = pickle.load(fout, encoding="latin1")    
    nmol = 0
    lengths = []
    reads_f = []
    labels = []
    holes = []
    mean_probs = []
    index = 0
    print(valid)
    for hole in tipds:
        if str(hole) not in valid: continue
        print(hole)
        read = tipds[int(hole)]
        if len(read) < min_len: continue
        if len(tipds[int(hole)]) >= length:
            reads_f.append(read[:length])
            lengths.append(len(tipds[int(hole)]))
        elif len(tipds[int(hole)]) < length:
            continue
        labels.append(valid[str(hole)])
        nmol += 1
        index += 1
        if samp != 0:
            if nmol == samp: break
    new_mat = np.vstack(reads_f)
    tipds = 0
    return (new_mat, lengths, labels)

rep1=0
length = 1000
k562_rep1_stam = '/avicenna/vramani/analyses/pacbio/pbrun_STAM/processed/binarized/pbrun_STAM_k562_chromatin_500U_Hia5_run1_bingmm'
k562_rep1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun_STAM/processed/onlyT/pbrun_STAM_k562_chromatin_500U_Hia5_run1_onlyT_zmwinfo'
valid_zmws = '/avicenna/vramani/analyses/pacbio/pbrun_STAM/ccs/total_zmws.txt'
rep1, l1, lbl = eat_pickle_filt("%s.pickle" % k562_rep1_stam, 'K562_Rep1', length, valid_zmws, samp = 250000)


In [ ]:
print(len(rep1))

In [ ]:
plt.figure(figsize=(20,20))
imshow(np.nan_to_num(rep1))
plt.show()

In [ ]:
def process_xcors(new_mat, max_len=1000, lengths=False):
    process = np.nan_to_num(pd.DataFrame(new_mat).rolling(33,axis=1, center=True, min_periods=1).mean())
    if isinstance(lengths, np.ndarray):
        auto_cor = xcor(process, process, maxlen=max_len, lengths=lengths)
    else:
        auto_cor = xcor(process, process, maxlen=max_len)
    process = None
    return auto_cor

def plotter(new_mat, auto_cor, clusters,smooth):
    fho = open('plot_stergachis_clusters.txt','w')
    for i in np.unique(clusters):
        print(i)
        mat_new = auto_cor[clusters == i]
        autocor_mean = np.nanmean(mat_new,axis=0)
        for j in range(len(autocor_mean)):
            print("%s\t%s\t%s" % (j, autocor_mean[j],i),file=fho)
        print(len(mat_new))
        mat_new2 = new_mat[clusters == i]
        mat_new2 = pd.DataFrame(mat_new2).rolling(smooth, axis=1, center=True, min_periods=1).mean()
        plt.figure(figsize=(30,10))
        subplot(121)
        plt.plot(range(len(mat_new[0])), np.nanmean(mat_new, axis=0))
        peak, data = signal.find_peaks(np.nanmean(mat_new, axis=0), height = 0.10, width = 25)
        if len(peak) == 0:
            print("undef")
        else:
            print(peak[0])
        xlabel('Offset (bp)')
        ylabel('Pearson\'s r')
        subplot(122)
        plt.plot(range(len(np.nanmean(mat_new2, axis=0))), np.nanmean(mat_new2, axis=0))
        xlabel('Distance to 5\'-MNase cut')
        ylabel('Average smoothed IPD (frames)')
        plt.show()
    fho.close()

# auto_cor = np.nan_to_num(process_xcors(rep1,max_len=5000))
# plt.figure(figsize=(20,20))
# imshow(np.nan_to_num(auto_cor))
# plt.show()
# mat = scanpy.AnnData(X=auto_cor)
# scanpy.tl.pca(mat)
# scanpy.pp.neighbors(mat,metric='correlation',n_neighbors=10)
# scanpy.tl.leiden(mat,resolution=0.4)
# clusters = np.array(mat.obs['leiden'])
plotter(np.array(rep1[:,:500]), auto_cor[:,:500], clusters, 33)


In [ ]:
print(len(lbl))

In [ ]:
#data frame of labels
total_dataframe = pd.DataFrame({'cluster_id': clusters, 'chromatin_type': lbl \
    })
total_dataframe

In [ ]:
import scipy as sp

# test = total_dataframe[(total_dataframe['chromatin_type'] == 'H3K9me3') | (total_dataframe['chromatin_type'] == 'H3K9me1')]['total_index']
# print(len(test))
# print(len(np.unique(test)))

fho = open('fishers_tests_fig_sterg.txt','w')
for i in np.unique(total_dataframe['cluster_id']):
    if i == '7': continue
    for j in np.unique(total_dataframe['chromatin_type']):
        num_clust_lab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] == j)])
        num_clust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] != j)])
        num_notclust_lab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] == j)])
        num_notclust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()

## Replicate analyses for the reviews (Replicate 1)

In [ ]:
import scipy as sp

def find_indices(sites, zmw_dict, samp_label, chrom_label):
    lines = open(sites)
    indices = []
    for entry in lines:
        split = entry.split()
        hole = "%s_%s" % (samp_label, split[0])
        if hole in zmw_dict:
            indices.append(zmw_dict[hole])
    return indices
        
#print(list(zmw2index.keys())[:100])
cluster_file = open('final_cluster_labels_total.txt')
lengths_total = np.load('length_filtered_lengths.npy')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)

#total_dataframe = pd.DataFrame({'cluster_id': [], 'chromatin_type': [], 'total_index': [], 'frag_length' : []})
cluster_ids = []
chromatin_types = []
total_index = []
frag_length = []

tfs = open('./pbrun4_purple/biology_analyses/new_beds.txt')
#tfs = ['coopfoots\n']

for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    print(sample)
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
#     sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
#     sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
    rep1_indices = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
#     rep2_indices = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
    rep3_indices = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
#     rep4_indices = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
#    tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
    tot_indices = np.concatenate((rep1_indices,rep3_indices))    
    cluster_indices = clusters[tot_indices]
    lengths_indices = lengths_total[tot_indices]
    tot_labels = np.repeat(tf, len(tot_indices))
    cluster_ids.append(cluster_indices)
    chromatin_types.append(tot_labels)
    frag_length.append(lengths_indices)
    total_index.append(tot_indices)

print(cluster_ids)    

#Also include Siva Satellite ZMWs
sites_rep1 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
#sites_rep2 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
sites_rep3 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
#sites_rep4 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
tf='Satellite'
rep1_indices = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
#rep2_indices = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
rep3_indices = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
#rep4_indices = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
#tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
tot_indices = np.concatenate((rep1_indices,rep3_indices))
cluster_indices = clusters[tot_indices]
lengths_indices = lengths_total[tot_indices]
tot_labels = np.repeat(tf, len(tot_indices))
cluster_ids.append(cluster_indices)
chromatin_types.append(tot_labels)
frag_length.append(lengths_indices)
total_index.append(tot_indices)

total_dataframe = pd.DataFrame({'cluster_id': np.concatenate(cluster_ids), 'chromatin_type': np.concatenate(chromatin_types) \
    , 'total_index': np.concatenate(total_index), 'frag_length' : np.concatenate(frag_length)})
total_dataframe

fho = open('fishers_tests_rep1.txt','w')
for i in np.unique(total_dataframe['cluster_id']):
    if i == '7': continue
    for j in np.unique(total_dataframe['chromatin_type']):
        num_clust_lab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] == j)])
        num_clust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] != j)])
        num_notclust_lab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] == j)])
        num_notclust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()

## Replicate 2

In [ ]:
import scipy as sp

def find_indices(sites, zmw_dict, samp_label, chrom_label):
    lines = open(sites)
    indices = []
    for entry in lines:
        split = entry.split()
        hole = "%s_%s" % (samp_label, split[0])
        if hole in zmw_dict:
            indices.append(zmw_dict[hole])
    return indices
        
#print(list(zmw2index.keys())[:100])
cluster_file = open('final_cluster_labels_total.txt')
lengths_total = np.load('length_filtered_lengths.npy')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)

#total_dataframe = pd.DataFrame({'cluster_id': [], 'chromatin_type': [], 'total_index': [], 'frag_length' : []})
cluster_ids = []
chromatin_types = []
total_index = []
frag_length = []

tfs = open('./pbrun4_purple/biology_analyses/new_beds.txt')
#tfs = ['coopfoots\n']

for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    print(sample)
#    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
#   sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_zmws' % sample
#    rep1_indices = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
    rep2_indices = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
#    rep3_indices = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
    rep4_indices = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
#    tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
    tot_indices = np.concatenate((rep2_indices,rep4_indices))    
    cluster_indices = clusters[tot_indices]
    lengths_indices = lengths_total[tot_indices]
    tot_labels = np.repeat(tf, len(tot_indices))
    cluster_ids.append(cluster_indices)
    chromatin_types.append(tot_labels)
    frag_length.append(lengths_indices)
    total_index.append(tot_indices)

print(cluster_ids)    

#Also include Siva Satellite ZMWs
#sites_rep1 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep2 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
#sites_rep3 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep4 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
tf='Satellite'
#rep1_indices = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
rep2_indices = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
#rep3_indices = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
rep4_indices = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
#tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
tot_indices = np.concatenate((rep2_indices,rep4_indices))
cluster_indices = clusters[tot_indices]
lengths_indices = lengths_total[tot_indices]
tot_labels = np.repeat(tf, len(tot_indices))
cluster_ids.append(cluster_indices)
chromatin_types.append(tot_labels)
frag_length.append(lengths_indices)
total_index.append(tot_indices)

total_dataframe = pd.DataFrame({'cluster_id': np.concatenate(cluster_ids), 'chromatin_type': np.concatenate(chromatin_types) \
    , 'total_index': np.concatenate(total_index), 'frag_length' : np.concatenate(frag_length)})
total_dataframe

fho = open('fishers_tests_rep2.txt','w')
for i in np.unique(total_dataframe['cluster_id']):
    if i == '7': continue
    for j in np.unique(total_dataframe['chromatin_type']):
        num_clust_lab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] == j)])
        num_clust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] != j)])
        num_notclust_lab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] == j)])
        num_notclust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()

In [ ]:
length = 500

tfs = open('./pbrun4_purple/biology_analyses/tfs.txt')
#tfs = ['GATA1\n']
lengths_total = np.array([0])
labels_total = np.array([0])
mat_total = []
replicate_tracker = []
for line in tfs:
    tf = line.split()[0]
    sample = '%s' % tf
    sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_flat_zmws' % sample
    cntrl_sites_rep1 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep2 = './pbrun4_purple/ccs/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep3 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    cntrl_sites_rep4 = './pbrun6/ccs/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.aligned.sorted.bam.%s_random_flat_zmws' % sample
    mat_rep1, lengths, labels = distill_tipds_flat_centered(rep1, sites_rep1, length, sample)
    
    cntrl_mat_rep1, cntrl_lengths, cntrl_labels = distill_tipds_flat_centered(rep1, cntrl_sites_rep1, length, "%s_control" % sample ,subsample=len(lengths))    
    
    mat_rep2, lengths2, labels2 = distill_tipds_flat_centered(rep2,  sites_rep2, length, sample)
    
    cntrl_mat_rep2, cntrl_lengths2, cntrl_labels2 = distill_tipds_flat_centered(rep2,  cntrl_sites_rep2, length, "%s_control" % sample,subsample=len(lengths2))    
    
    mat_rep3, lengths3, labels3 = distill_tipds_flat_centered(rep3,  sites_rep3, length, sample)
    
    cntrl_mat_rep3, cntrl_lengths3, cntrl_labels3 = distill_tipds_flat_centered(rep3,cntrl_sites_rep3, length, "%s_control" % sample,subsample=len(lengths3))    
    
    mat_rep4, lengths4, labels4 = distill_tipds_flat_centered(rep4, sites_rep4, length, sample)
    
    cntrl_mat_rep4, cntrl_lengths4, cntrl_labels4 = distill_tipds_flat_centered(rep4, cntrl_sites_rep4, length, "%s_control" % sample,subsample=len(lengths4))    
    
#    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4,cntrl_mat_rep1,cntrl_mat_rep2,cntrl_mat_rep3,cntrl_mat_rep4))
#    mat_total.extend((mat_rep1,mat_rep2,mat_rep3,mat_rep4))
#    mat_msix = np.vstack((mat_rep1, mat_rep2, mat_rep3, mat_rep4))
#    mat_msix_control = np.vstack((cntrl_mat_rep1, cntrl_mat_rep2, cntrl_mat_rep3, cntrl_mat_rep4))
    for i in range(len(mat_rep1)):
        replicate_tracker.append('Replicate 1')
    for i in range(len(mat_rep2)):
        replicate_tracker.append('Replicate 2')
    for i in range(len(mat_rep3)):
        replicate_tracker.append('Replicate 1')
    for i in range(len(mat_rep4)):
        replicate_tracker.append('Replicate 2')
    for i in range(len(cntrl_mat_rep1)):
        replicate_tracker.append('Replicate 1')
    for i in range(len(cntrl_mat_rep2)):
        replicate_tracker.append('Replicate 2')
    for i in range(len(cntrl_mat_rep3)):
        replicate_tracker.append('Replicate 1')
    for i in range(len(cntrl_mat_rep4)):
        replicate_tracker.append('Replicate 2')
    print("%s\t%s" % (sample, (len(mat_rep1)+len(mat_rep2)+len(mat_rep3)+len(mat_rep4))))
    print("%s_control\t%s" % (sample, (len(cntrl_mat_rep1)+len(cntrl_mat_rep2)+len(cntrl_mat_rep3)+len(cntrl_mat_rep4))))
    lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4, cntrl_lengths, cntrl_lengths2,cntrl_lengths3,cntrl_lengths4))
    labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4,cntrl_labels, cntrl_labels2, cntrl_labels3, cntrl_labels4))
#     lengths_total = np.concatenate((lengths_total, lengths, lengths2,lengths3,lengths4))
#     labels_total = np.concatenate((labels_total, labels, labels2,labels3,labels4))
#     process = pd.Series(np.nanmean(mat_msix, axis=0)).rolling(33,center=True,min_periods=1).mean()
#     process_neg = pd.Series(np.nanmean(mat_msix_control, axis=0)).rolling(33,center=True,min_periods=1).mean()
#     process = np.nanmean(binarize_sig(pd.DataFrame(mat_msix).rolling(10, axis=1, center=True, min_periods=1).mean()), axis=0)
#     process_neg = np.nanmean(binarize_sig(pd.DataFrame(mat_msix_control).rolling(10, axis=1, center=True, min_periods=1).mean()), axis=0)
#     plt.figure(figsize=(10,10))
#     subplot(121)
#     plt.plot(range(len(process)), process)
#     subplot(122)
#     plt.plot(range(len(process_neg)), process_neg)
#     plt.show()
#     fho=open('%s_m6A.txt' % tf,'w')
#     for i in range(len(process)):
#         print("%s\t%s\t%s\tsite" % (i, process[i], tf),file=fho)
#         print("%s\t%s\t%s\tcontrol" % (i, process_neg[i], tf),file=fho)
#     fho.close()

mat_total = np.vstack(mat_total)
lengths_total = lengths_total[1:]
labels_total = labels_total[1:]

tfs.close()

#np.save('length_filtered_molecules_tfs',mat_total)

In [ ]:
cluster_file = open('final_cluster_labels_tfs.txt')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[0])
clusters = np.array(clusters)
cluster_file.close()
print(len(labels_total))
print(len(replicate_tracker))
print(len(clusters))
fho = open('repro_tfs_rep1.txt','w')
#Replicate 1
cluster_df = {'clusters':clusters, 'labels':labels_total[1:], 'replicates':replicate_tracker}
cluster_df = pd.DataFrame(cluster_df)
cluster_df = cluster_df[cluster_df['replicates'] == 'Replicate 1']
for i in np.unique(clusters):
    for j in np.unique(labels_total[1:]):
        num_clust_lab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] == j)])
        num_clust_notlab = len(cluster_df[(cluster_df['clusters'] == i) & (cluster_df['labels'] != j)])
        num_notclust_lab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] == j)])
        num_notclust_notlab = len(cluster_df[(cluster_df['clusters'] != i) & (cluster_df['labels'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()
#labs = np.a

## Separate out alpha, beta, and gamma satellite sequences

In [ ]:
import scipy as sp

def find_indices(sites, zmw_dict, samp_label, chrom_label):
    lines = open(sites)
    indices = []
    labels = []
    for entry in lines:
        split = entry.split()
        hole = "%s_%s" % (samp_label, split[0])
        if hole in zmw_dict:
            indices.append(zmw_dict[hole])
            if split[4][0] == 'A':
                labels.append('alpha')
            elif split[4][0] =='B':
                labels.append('beta')
            elif split[4][0] == 'G':
                labels.append('gamma')
    return (indices, labels)
        
#print(list(zmw2index.keys())[:100])
cluster_file = open('final_cluster_labels_total.txt')
lengths_total = np.load('length_filtered_lengths.npy')
clusters = []
for line in cluster_file:
    split = line.split()
    clusters.append(split[1])
clusters = np.array(clusters)

#total_dataframe = pd.DataFrame({'cluster_id': [], 'chromatin_type': [], 'total_index': [], 'frag_length' : []})
cluster_ids = []
chromatin_types = []
total_index = []
frag_length = []

 

#Also include Siva Satellite ZMWs
sites_rep1 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin--k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep2 = './pbrun4_purple/biology_analyses/Purple_PBRun.k562_postlyso_chromatin_rep2--k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
sites_rep3 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin.ccs.blast.satellite.txt.zmws'
sites_rep4 = './pbrun4_purple/biology_analyses/pbrun6.split.k562_postlyso_chromatin_rep2.ccs.blast.satellite.txt.zmws'
tfs = []

rep1_indices,labels1 = find_indices(sites_rep1, zmw2index, 'K562_Rep1', tf)
rep2_indices,labels2 = find_indices(sites_rep2, zmw2index, 'K562_Rep2', tf)
rep3_indices,labels3 = find_indices(sites_rep3, zmw2index, 'K562_Rep3', tf)
rep4_indices,labels4 = find_indices(sites_rep4, zmw2index, 'K562_Rep4', tf)
tot_indices = np.concatenate((rep1_indices,rep2_indices,rep3_indices,rep4_indices))    
cluster_indices = clusters[tot_indices]
lengths_indices = lengths_total[tot_indices]
tot_labels = np.concatenate((labels1,labels2,labels3,labels4))
cluster_ids.append(cluster_indices)
chromatin_types.append(tot_labels)
frag_length.append(lengths_indices)
total_index.append(tot_indices)

total_dataframe = pd.DataFrame({'cluster_id': np.concatenate(cluster_ids), 'chromatin_type': np.concatenate(chromatin_types) \
    , 'total_index': np.concatenate(total_index), 'frag_length' : np.concatenate(frag_length)})
total_dataframe

fho = open('fishers_tests_satellite.txt','w')
for i in np.unique(total_dataframe['cluster_id']):
    if i == '7': continue
    for j in np.unique(total_dataframe['chromatin_type']):
        num_clust_lab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] == j)])
        num_clust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] == i) & (total_dataframe['chromatin_type'] != j)])
        num_notclust_lab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] == j)])
        num_notclust_notlab = len(total_dataframe[(total_dataframe['cluster_id'] != i) & (total_dataframe['chromatin_type'] != j)])
        odds_r, pval = sp.stats.fisher_exact([[num_clust_lab, num_notclust_lab], \
            [num_clust_notlab, num_notclust_notlab]])
        print("%s\t%s\t%s\t%s\t%s" % (i, j, num_clust_lab, odds_r, pval), file=fho)
fho.close()

## New data analyses

In [ ]:
length = 1000

fho = open('for_supplement_sig.txt','w')
fho2 = open('for_supplement_lens.txt','w')

##BINARIZED
k562_rep3_long1 = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/binarized/pbrun8_berkeley_k562_4C_10m_plusM_rep1_bingmm'
k562_rep4_long1 = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/binarized/pbrun8_berkeley_k562_4C_10m_plusM_rep2_bingmm'
k562_rep3_long1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/onlyT/pbrun8_berkeley_k562_4C_10m_plusM_rep1_onlyT'
k562_rep4_long1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/onlyT/pbrun8_berkeley_k562_4C_10m_plusM_rep2_onlyT'

k562_rep3_long2 = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/binarized/pbrun8_cell2_k562_4C_10m_plusM_rep1_bingmm'
k562_rep4_long2 = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/binarized/pbrun8_cell2_k562_4C_10m_plusM_rep2_bingmm'
k562_rep3_long2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/onlyT/pbrun8_cell2_k562_4C_10m_plusM_rep1_onlyT'
k562_rep4_long2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/onlyT/pbrun8_cell2_k562_4C_10m_plusM_rep2_onlyT'

rep1, l1, lb1, h1 = eat_pickle_b2m("%s.pickle" % k562_rep3_long1, 'K562_Rep1', length)
rep2, l2, lb2, h2 = eat_pickle_b2m("%s.pickle" % k562_rep4_long1, 'K562_Rep2', length)
rep3, l3, lb3, h3 = eat_pickle_b2m("%s.pickle" % k562_rep3_long2, 'K562_Rep3', length)
rep4, l4, lb4, h4 = eat_pickle_b2m("%s.pickle" % k562_rep4_long2, 'K562_Rep4', length)
mat_total=np.vstack((rep1,rep2,rep3,rep4))
lengths_total = np.array(np.concatenate((l1,l2,l3,l4), axis=None))
# #lb_total = np.array(np.concatenate((lb1,lb2,lb3,lb4), axis=None))
# h_total = np.array(np.concatenate((h1,h2,h3,h4), axis=None))
# rep1, rep2, rep3, rep4 = [None, None, None, None]
plotme_sig = np.nanmean(mat_total,axis=0)

for i in range(len(plotme_sig)):
    print("%s\t%s\t4deg_10min" % (i, plotme_sig[i]),file=fho)

for i in lengths_total:
    print("%s\t4deg_10min" % i, file=fho2)

plt.figure(figsize=(10,10))
plt.plot(range(len(mat_total[0])), np.nanmean(mat_total,axis=0))
plt.show()


In [ ]:
length = 1000


# #BINARIZED
k562_rep3_long1 = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/binarized/pbrun8_berkeley_k562_37C_1m_plusM_rep1_bingmm'
#k562_rep4_long1 = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/binarized/pbrun8_berkeley_k562_37C_1m_plusM_rep2_bingmm'
k562_rep3_long1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/onlyT/pbrun8_berkeley_k562_37C_1m_plusM_rep1_onlyT'
#k562_rep4_long1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/onlyT/pbrun8_berkeley_k562_37C_1m_plusM_rep2_onlyT'


k562_rep3_long2 = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/binarized/pbrun8_cell2_k562_37C_1m_plusM_rep1_bingmm'
#k562_rep4_long2 = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/binarized/pbrun8_cell2_k562_37C_1m_plusM_rep2_bingmm'
k562_rep3_long2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/onlyT/pbrun8_cell2_k562_37C_1m_plusM_rep1_onlyT'
#k562_rep4_long2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/onlyT/pbrun8_cell2_k562_37C_1m_plusM_rep2_onlyT'

rep1, l1, lb1, h1 = eat_pickle_b2m("%s.pickle" % k562_rep3_long1, 'K562_Rep1', length)
#rep2, l2, lb2, h2 = eat_pickle_b2m("%s.pickle" % k562_rep4_long1, 'K562_Rep2', length)
rep3, l3, lb3, h3 = eat_pickle_b2m("%s.pickle" % k562_rep3_long2, 'K562_Rep3', length)
#rep4, l4, lb4, h4 = eat_pickle_b2m("%s.pickle" % k562_rep4_long2, 'K562_Rep4', length)
mat_total=np.vstack((rep1,rep3))
lengths_total = np.array(np.concatenate((l1,l3), axis=None))
# #lb_total = np.array(np.concatenate((lb1,lb2,lb3,lb4), axis=None))
# h_total = np.array(np.concatenate((h1,h2,h3,h4), axis=None))
# rep1, rep2, rep3, rep4 = [None, None, None, None]
print(mat_total)
plotme_sig = np.nanmean(mat_total,axis=0)

for i in range(len(plotme_sig)):
    print("%s\t%s\t37deg_1min" % (i, plotme_sig[i]),file=fho)

for i in lengths_total:
    print("%s\t37deg_1min" % i, file=fho2)

plt.figure(figsize=(10,10))
plt.plot(range(len(mat_total[0])), np.nanmean(mat_total,axis=0))
plt.show()

plt.figure(figsize=(10,10))
sns.distplot(lengths_total)
plt.show()

In [ ]:
length = 1000


# #BINARIZED
k562_rep3_long1 = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/binarized/pbrun8_berkeley_k562_4C_60m_plusM_rep1_bingmm'
#k562_rep4_long1 = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/binarized/pbrun8_berkeley_k562_37C_1m_plusM_rep2_bingmm'
k562_rep3_long1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/onlyT/pbrun8_berkeley_k562_4C_60m_plusM_rep1_onlyT'
#k562_rep4_long1_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_berkeley/processed/onlyT/pbrun8_berkeley_k562_37C_1m_plusM_rep2_onlyT'


k562_rep3_long2 = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/binarized/pbrun8_cell2_k562_4C_60m_plusM_rep1_bingmm'
k562_rep4_long2 = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/binarized/pbrun8_cell2_k562_4C_60m_plusM_rep2_bingmm'
k562_rep3_long2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/onlyT/pbrun8_cell2_k562_4C_60m_plusM_rep1_onlyT'
k562_rep4_long2_zmwinfo = '/avicenna/vramani/analyses/pacbio/pbrun8_cell2/processed/onlyT/pbrun8_cell2_k562_4C_60m_plusM_rep2_onlyT'

rep1, l1, lb1, h1 = eat_pickle_b2m("%s.pickle" % k562_rep3_long1, 'K562_Rep1', length)
#rep2, l2, lb2, h2 = eat_pickle_b2m("%s.pickle" % k562_rep4_long1, 'K562_Rep2', length)
rep3, l3, lb3, h3 = eat_pickle_b2m("%s.pickle" % k562_rep3_long2, 'K562_Rep3', length)
#rep4, l4, lb4, h4 = eat_pickle_b2m("%s.pickle" % k562_rep4_long2, 'K562_Rep4', length)
mat_total=np.vstack((rep1,rep3,rep4))
lengths_total = np.array(np.concatenate((l1,l3,l4), axis=None))
# #lb_total = np.array(np.concatenate((lb1,lb2,lb3,lb4), axis=None))
# h_total = np.array(np.concatenate((h1,h2,h3,h4), axis=None))
# rep1, rep2, rep3, rep4 = [None, None, None, None]
print(mat_total)
plotme_sig = np.nanmean(mat_total,axis=0)

for i in range(len(plotme_sig)):
    print("%s\t%s\t4deg_60min" % (i, plotme_sig[i]),file=fho)

for i in lengths_total:
    print("%s\t4deg_60min" % i, file=fho2)

plt.figure(figsize=(10,10))
plt.plot(range(len(mat_total[0])), np.nanmean(mat_total,axis=0))
plt.show()

plt.figure(figsize=(10,10))
sns.distplot(lengths_total)
plt.show()

fho.close()
fho2.close()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(range(len(mat_total[0])), np.nanmean(mat_total,axis=0))
plt.show()

In [ ]:
fho = open('4deg_10min_lengths.txt','w')
for i in range(len(lengths_total)):
    print(lengths_total[i], file=fho)
fho.close()
    